In [ ]:
#### We will use the old method relying on underriting model in metabase to pull Lusa scores here

## Change1 : Changed oull of Lusa v1 to Lusa v2
## Change2 : Changing Provision Rates file for Lusa v2
## Change3 : Making changes on functions to be compatible with Lusa v2

In [1]:
import numpy as np
import pandas as pd

import belomonte
conn = belomonte.connect("aakash.katdare")

2017-12-18T17:31:03-0200 [MainProcess] INFO botocore.credentials Found credentials in environment variables.
2017-12-18T17:31:03-0200 [MainProcess] INFO botocore.vendored.requests.packages.urllib3.connectionpool Starting new HTTPS connection (1): redshift.us-east-1.amazonaws.com


In [2]:
# Importing provisions for Lusa v2
provision_table = pd.read_csv('lusa_v2_provisions.csv')
provision_table= provision_table[['days_late','lusa_v2_band','mth','lusa_v2_bin_25','provision_rate']]
provision_table['provision_rate']=provision_table.provision_rate.fillna(0)
provision_table['provision_rate_mult']=provision_table['provision_rate']*1000000

In [3]:
import nossa_senhora
#provision_table = pd.read_csv('lusa_v2_provisions.csv')
nossa_senhora.do_belomonte.upload_temp_table(conn, provision_table, temp_table = 'provision_table2')

Just Finished 0 rows of 140999 rows
Just Finished 50000 rows of 140999 rows
Just Finished 100000 rows of 140999 rows
Finished uploading temporary table


In [4]:
all_balances = pd.read_sql("""(WITH
    -- Release Dates of Accounts pulled here

        released_accounts AS (
          SELECT balances.account__id,    min(balances.date::date) AS released_date, (date_part('month', min(balances.date::date)) + 100*date_part('year', min(balances.date::date))) AS Cohort FROM origin.balances GROUP BY balances.account__id),

    -- Non-Balance performance variables here e.g. Spend, new recv, lossest, fx spread, interest revenue etc. of Accounts pulled here

       deltas AS (SELECT

                     deltas.account__id,
                     deltas.date::date,
                     greatest(asset__revolving + asset__late + asset__future_installments_issuer_interest + asset__installments_issuer,0) as sum_positive_mov,
                     (liability__payable_brazil + liability__payable_foreign + liability__payable_installments) AS gmv,
                     (liability__payable_brazil + liability__payable_foreign + profit_and_loss__interchange_brazil + profit_and_loss__interchange_foreign) AS igmv,
                     (liability__payable_brazil + profit_and_loss__interchange_brazil) AS igmv_domestic,
                     (liability__payable_foreign + profit_and_loss__interchange_foreign) AS gmv_foreign,
                     (liability__payable_brazil + liability__payable_installments) AS gmv_domestic,
                     (profit_and_loss__installments_issuer_interest + profit_and_loss__revolving_interest + profit_and_loss__late_interest) AS interest_revenue,
                     (profit_and_loss__installments_issuer_interest) AS interest_revenue_installments,
                     (profit_and_loss__revolving_interest) AS interest_revenue_normal,
                     --(profit_and_loss__forex_loss + profit_and_loss__forex_gain + profit_and_loss__fx_spread) AS fx_spread,
                     (profit_and_loss__fx_spread) AS fx_spread,
                     (profit_and_loss__identity_fraud_losses) AS identity_fraud_losses,
                     (profit_and_loss__fraud_losses) AS fraud_losses,
                     (profit_and_loss__credit_loss) AS credit_loss,
                     (profit_and_loss__credit_loss_recovery) as credit_loss_recovery,
                     (profit_and_loss__late_fee) as late_fee,
                     (profit_and_loss__subscription_fee) as rewards_fees
                   FROM origin.deltas
                   ),

    -- Account Lusa scores (first record)


    Lusas AS
        (select account__id, lusa_v2_bin_25 as Lusa_band
            FROM dimension.prospect
        JOIN origin.account_key_integrity as k
            ON k.prospect__id = dimension.prospect.prospect__id
        --WHERE dimension.prospect.applied_at > '2017-03-01' 
    ),



    -- Merging Cohort and Lusa with performance

                        Deltas_releases AS (SELECT

                          deltas.account__id, released_date,
                          deltas.date::date,
                          sum_positive_mov, gmv, igmv, igmv_domestic, gmv_foreign, gmv_domestic, interest_revenue, interest_revenue_installments,
                          interest_revenue_normal, fx_spread, identity_fraud_losses, fraud_losses, credit_loss, credit_loss_recovery, late_fee, rewards_fees

                                   FROM deltas
                                   JOIN released_accounts ON deltas.account__id = released_accounts.account__id
                                   JOIN Lusas ON deltas.account__id = Lusas.account__id
                                  WHERE ((deltas.date::date - released_date) / 30) < ((CURRENT_DATE - released_date) / 30)
                                  ),



    -- Avg daily Balances of accounts by month

        Bals AS (SELECT
                      balances.account__id,
                      balances.date::date,
                      released_date,
                      (asset__closed + asset__settled_brazil + asset__settled_foreign + asset__unsettled) as adb_float_p,
                      (asset__settled_financed) as adb_float_set_fin,
                      (liability__payable_brazil + liability__payable_foreign) as adb_float_n,
                      (asset__revolving) as adb_revolving,
                      (asset__late) as adb_late,
                      (asset__revolving + asset__late + asset__installments_merchant + asset__installments_issuer + asset__future_installments_issuer_interest + asset__chargeback + asset__closed + asset__settled_brazil + asset__settled_financed + asset__settled_foreign + asset__future_installments_issuer_iof + asset__pre_loss + asset__renegotiated + asset__due_installments + asset__interests_fees_and_taxes) AS avg_outstanding_bal,
                      (asset__future_installments_issuer_interest) as adb_issuer_interest,
                      (asset__interests_fees_and_taxes) as adb_other,
                      (asset__installments_issuer + asset__future_installments_issuer_iof + asset__future_installments_issuer_interest + asset__settled_financed) as adb_installments_issuer,
                      (asset__revolving + asset__late + asset__future_installments_issuer_interest + asset__installments_issuer + asset__future_installments_issuer_iof + asset__renegotiated + asset__settled_financed + asset__due_installments + asset__interests_fees_and_taxes ) as adb,
                      Cohort,
                      Lusa_band

                      FROM origin.balances
                      JOIN released_accounts ON balances.account__id = released_accounts.account__id
                      JOIN Lusas ON balances.account__id = Lusas.account__id
                      WHERE ((balances.date::date - released_date) / 30) < ((CURRENT_DATE - released_date) / 30)
                      ),


    -- All performance merged

                    Deltas_Bals_releases AS( Select bals.account__id, bals.date, bals.released_date,
                                            sum_positive_mov, gmv, igmv, igmv_domestic, gmv_foreign, gmv_domestic, interest_revenue, interest_revenue_installments,
                                            interest_revenue_normal, fx_spread, identity_fraud_losses, fraud_losses, credit_loss, credit_loss_recovery, late_fee, rewards_fees, Cohort, Lusa_band,
                                            adb_float_p, adb_float_set_fin, adb_float_n, adb_revolving, adb_late, adb_issuer_interest, adb_installments_issuer, avg_outstanding_bal,
                                            adb, adb_other
                                            FROM Bals
                                            LEFT JOIN Deltas_releases ON Deltas_releases.account__id = Bals.account__id AND Deltas_releases.date = Bals.date
                                            ),

    -- All performance merged grouped at a higher level


                    Perf_acc_lvl AS (Select deltas_Bals_releases.account__id,
                                               (deltas_Bals_releases.date - deltas_Bals_releases.released_date) / 30 AS Mth,
                                               sum(sum_positive_mov) as New_balance_added,
                                               sum(gmv) as purchase_f_gmv,
                                               sum(igmv) as purchase_f_igmv,
                                               sum(igmv_domestic) as purchase_f_igmv_domestic,
                                               sum(gmv_foreign) as purchase_f_gmv_foreign,
                                               sum(gmv_domestic) as purchase_f_gmv_domestic,
                                               sum(interest_revenue) as int_f_revenue,
                                               sum(interest_revenue_installments) as int_installments_f_revenue,
                                               sum(interest_revenue_normal) as int_f_revenue_normal,
                                               sum(fx_spread) as fx_spread_f,
                                               sum(identity_fraud_losses) as id_frd_f,
                                               sum(fraud_losses) as frd_f,
                                               sum(credit_loss) as credit_loss_f,
                                               sum(credit_loss_recovery) as credit_loss_recovery_f,
                                               sum(late_fee) as late_fee_f,
                                               sum(rewards_fees) as rewards_fee_f,
                                               avg(adb_float_p) as positive_float_adb,
                                               avg(adb_float_set_fin) as float_adb_fin,
                                               avg(adb_float_n) as negative_float_adb,
                                               avg(adb_revolving) as revolve_adb,
                                               avg(avg_outstanding_bal) as avg_outstanding_bal,
                                               avg(adb_late) as late_adb,
                                               avg(adb_issuer_interest) as issuer_int_adb,
                                               avg(adb_installments_issuer) as issuer_installments_adb,
                                               avg(adb) as adb_overall,
                                               avg(adb_other) as adb_other,
                                               Cohort, Lusa_band

                                               FROM Deltas_Bals_releases
                                               GROUP BY account__id, mth, Cohort, lusa_band
                                               ORDER BY account__id, mth, Cohort, lusa_band
                                               ),


    -- Activity flags by account_id and month since release


                       Activity_flags AS(    SELECT Deltas_releases.account__id,
                                        (Deltas_releases.date - released_date) / 30 AS Mth,
                                        (CASE when sum(gmv) > 0 then 1 else 0 end) as Purchase_active_Rate,
                                        (CASE when sum(interest_revenue) > 0 then 1 else 0 end) as Balance_active_Rate

                                            FROM Deltas_releases
                                            GROUP BY Account__id, mth
                                            Order BY account__id, mth
                                        ),


    -- DQ statuses by month

                       DQall AS(
                         select
                         date.date::date,
                         released_date,
                         delinquency_period_start,
                         date.date::date - released_date as days_since_release,
                         date.date::date - delinquency_period_start as days_delinquent,
                         (CASE when (1 + ((date.date::date - delinquency_period_start) / 30)) > 7 then 7 else (1 + ((date.date::date - delinquency_period_start) / 30)) end) as DQ_stat,
                         (((date.date::date - released_date) / 30) - 1) AS Mth,
                         delinquency_table.account__id
                         from origin.delinquency_table
                         join dimension.date on date.date::date between delinquency_period_start and coalesce(delinquency_period_end, CURRENT_DATE)
                         join released_accounts on delinquency_table.account__id = released_accounts.account__id

                         ),

                         DQS AS(Select DQall.account__id, DQall.mth, DQall.DQ_Stat, days_delinquent from DQall where mod((DQall.date - DQall.released_date),30) = 0),


    -- Month of Charge-off

                         Chargeoffmth AS(
                         Select DQall.account__id,
                         min(DQall.mth) + 1 as Chargeoff_mth,
                         min((DQall.delinquency_period_start - released_date) / 30) as chargeoff_bill_mth
                         from DQall
                         Where DQall.days_delinquent = 180
                         Group by account__id
                         Order by account__id
                         ),


    -- Outstanding Balance every month to calculate provisions

                         outstanding_bals AS(
                         Select
                         balances.account__id,
                         (asset__revolving + asset__late + asset__installments_merchant + asset__installments_issuer + asset__future_installments_issuer_interest + asset__chargeback + asset__closed + asset__settled_brazil + asset__settled_financed + asset__settled_foreign + asset__future_installments_issuer_iof + asset__pre_loss + asset__renegotiated + asset__due_installments + asset__interests_fees_and_taxes) AS outstanding_balance,
                         asset__max_limit as max_credit_limit,
                         (asset__renegotiated + asset__due_installments + asset__revolving + asset__late + asset__installments_merchant + asset__installments_issuer + asset__future_installments_issuer_interest + asset__chargeback + asset__closed + asset__settled_brazil + asset__settled_financed + asset__settled_foreign + asset__future_installments_issuer_iof + asset__pre_loss - profit_and_loss__credit_loss + asset__interests_fees_and_taxes) AS chgoffamt_f,
                         (-profit_and_loss__credit_loss) as chg_crl,
                         (((balances.date - released_date) / 30) - 1) AS Mth

                         from origin.balances
                         join released_accounts on balances.account__id = released_accounts.account__id
                         where mod((balances.date - released_date),30) = 0
                         ),


    -- Merging all

                                          ACCT_mth_lvl_summary AS (SELECT Perf_acc_lvl.account__id, Perf_acc_lvl.mth, cohort, lusa_band,
                                                            New_balance_added, purchase_f_gmv, purchase_f_igmv, purchase_f_igmv_domestic, purchase_f_gmv_foreign, purchase_f_gmv_domestic,
                                                            int_f_revenue, int_installments_f_revenue, int_f_revenue_normal, fx_spread_f, id_frd_f, frd_f, credit_loss_f, credit_loss_recovery_f,
                                                            positive_float_adb, float_adb_fin, negative_float_adb, revolve_adb, late_adb, issuer_int_adb, avg_outstanding_bal,
                                                            issuer_installments_adb, adb_overall, adb_other, outstanding_balance, max_credit_limit, late_fee_f, rewards_fee_f,
                                                            days_delinquent,
                                                            (provision_rate_mult * outstanding_balance / 1000000)::float as provision_balance,
                                                            Activity_flags.Purchase_active_rate,
                                                            Activity_flags.Balance_active_rate,

                                                            (CASE   when DQS.DQ_Stat is null then 0
                                                                    else DQS.DQ_Stat end) as DQ_stat,

                                                            (CASE   when Chargeoffmth.Chargeoff_mth is null then 0
                                                                    else 1 end) as CO_stat,

                                                            (CASE   when Chargeoffmth.Chargeoff_bill_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_bill_mth > Perf_acc_lvl.mth then 0
                                                                    else  1 end) as CO_now,

                                                            (CASE   when Chargeoffmth.Chargeoff_bill_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_bill_mth = Perf_acc_lvl.mth then 1
                                                                    else  0 end) as CO_this_mth,

                                                            (CASE   when Chargeoffmth.Chargeoff_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_mth = Perf_acc_lvl.mth then 1
                                                                    else  0 end) as CO_this_mth_Accounting,

                                                            (CASE   when Chargeoffmth.Chargeoff_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_mth <= Perf_acc_lvl.mth then chgoffamt_f
                                                                    else  0 end) as chg,
                                                                    
                                                            chg_crl,
                                                            
                                                            greatest(chg_crl,
                                                            (CASE   when Chargeoffmth.Chargeoff_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_mth <= Perf_acc_lvl.mth then chgoffamt_f
                                                                    else  0 end)
                                                            ) as great_crl_loss,
                                                            
                                                            (greatest(chg_crl,
                                                            (CASE   when Chargeoffmth.Chargeoff_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_mth <= Perf_acc_lvl.mth then chgoffamt_f
                                                                    else  0 end)
                                                            ))
                                                             
                                                             - 
                                                             
                                                             
                                                             (sum(greatest(chg_crl,
                                                            (CASE   when Chargeoffmth.Chargeoff_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_mth <= Perf_acc_lvl.mth then chgoffamt_f
                                                                    else  0 end)
                                                            )) 
                                                             
                                                             over (partition by Perf_acc_lvl.account__id order by Perf_acc_lvl.mth rows between 1 preceding and 1 preceding))
                                                             
                                                            as chargeoffamt_f

                                                            FROM Perf_acc_lvl
                                                            LEFT JOIN Activity_flags ON Perf_acc_lvl.account__id = Activity_flags.account__id AND Perf_acc_lvl.mth = Activity_flags.mth
                                                            LEFT JOIN DQS ON Perf_acc_lvl.account__id = DQS.account__id AND Perf_acc_lvl.mth = DQS.mth
                                                            LEFT JOIN Chargeoffmth ON Perf_acc_lvl.account__id = Chargeoffmth.account__id
                                                            LEFT JOIN outstanding_bals ON Perf_acc_lvl.account__id = outstanding_bals.account__id AND Perf_acc_lvl.mth = outstanding_bals.mth
                                                            LEFT JOIN provision_table2 ON Perf_acc_lvl.mth = provision_table2.mth AND Perf_acc_lvl.lusa_band = provision_table2.lusa_v2_bin_25 AND days_delinquent + 1 = provision_table2.days_late
                                                            ),
                                                            
                                                            
                                                            


    -- Summarizing at a higher level

                     cohort_lvl_summary As (Select
                     mth,
                     cohort,
                     lusa_band,
                     DQ_stat,
                     CO_stat,
                     CO_now,
                     CO_this_mth,
                     CO_this_mth_Accounting,
                     Purchase_active_rate,
                     Balance_active_rate,

                     count(account__id) as Cust,
                     sum(days_delinquent) as days_delinquent,
                     sum(New_balance_added) as New_balance_added,
                     sum(purchase_f_gmv) as purchase_f_gmv,
                     sum(purchase_f_igmv) as purchase_f_igmv,
                     sum(purchase_f_igmv_domestic) as purchase_f_igmv_domestic,
                     sum(purchase_f_gmv_foreign) as purchase_f_gmv_foreign,
                     sum(purchase_f_gmv_domestic) as purchase_f_gmv_domestic,
                     sum(int_f_revenue) as int_f_revenue,
                     sum(int_installments_f_revenue) as int_installments_f_revenue,
                     sum(int_f_revenue_normal) as int_f_revenue_normal,
                     sum(fx_spread_f) as fx_spread_f,
                     sum(id_frd_f) as id_frd_f,
                     sum(frd_f) as frd_f,
                     sum(credit_loss_f) as credit_loss_f,
                     sum(credit_loss_recovery_f) as credit_loss_recovery_f,
                     sum(late_fee_f) as late_fee_f,
                     sum(rewards_fee_f) as rewards_fee_f,
                     sum(positive_float_adb) as positive_float_adb,
                     sum(float_adb_fin) as float_adb_fin,
                     sum(negative_float_adb) as negative_float_adb,
                     sum(revolve_adb) as revolve_adb,
                     sum(late_adb) as late_adb,
                     sum(issuer_int_adb) as issuer_int_adb,
                     sum(issuer_installments_adb) as issuer_installments_adb,
                     sum(adb_overall) as adb_overall,
                     sum(adb_other) as adb_other,
                     sum(outstanding_balance) as outstanding_balance,
                     sum(provision_balance) as provision_balance,
                     sum(avg_outstanding_bal) as avg_outstanding_bal,
                     sum(chargeoffamt_f) as chargeoffamt_f,
                     sum(max_credit_limit) as max_credit_limit

                     from ACCT_mth_lvl_summary
                     GROUP BY mth, cohort, lusa_band, DQ_stat, CO_stat, CO_now, CO_this_mth, CO_this_mth_Accounting, Purchase_active_rate, Balance_active_rate
                     ORDER BY mth, cohort, lusa_band, DQ_stat, CO_stat, CO_now, CO_this_mth, CO_this_mth_Accounting, Purchase_active_rate, Balance_active_rate
                      ),


    -- Enabling certain Exclusions

                      excludes AS (
                      select max(mth) as exclude_censor_mth,
                      max(mth) - 6 as exclude_Chargeoff_mth,
                      cohort
                      from cohort_lvl_summary group by cohort)


    -- Final pull to bring all together

         Select mth,
                     cohort_lvl_summary.cohort,
                     lusa_band,
                     DQ_stat,
                     CO_stat,
                     CO_now,
                     CO_this_mth,
                     CO_this_mth_Accounting,
                     Purchase_active_rate,
                     Balance_active_rate,

                     Cust,
                     days_delinquent,
                     New_balance_added,
                     purchase_f_gmv,
                     purchase_f_igmv,
                     purchase_f_igmv_domestic,
                     purchase_f_gmv_foreign,
                     purchase_f_gmv_domestic,
                     int_f_revenue,
                     int_installments_f_revenue,
                     int_f_revenue_normal,
                     fx_spread_f,
                     id_frd_f,
                     frd_f,
                     credit_loss_f,
                     credit_loss_recovery_f,
                     late_fee_f,
                     rewards_fee_f,
                     positive_float_adb,
                     float_adb_fin,
                     negative_float_adb,
                     revolve_adb,
                     late_adb,
                     issuer_int_adb,
                     issuer_installments_adb,
                     adb_other,
                     adb_overall,
                     outstanding_balance,
                     provision_balance,
                     avg_outstanding_bal,
                     max_credit_limit,
                     chargeoffamt_f,
                     (CASE when exclude_censor_mth <= mth then 1 else 0 end) as exclude_censor,
                     (CASE when exclude_chargeoff_mth <= mth then 1 else 0 end) as exclude_CO
                     from cohort_lvl_summary LEFT JOIN excludes ON cohort_lvl_summary.cohort = excludes.cohort




)                     
""",conn)


In [5]:
all_balances.shape

(544765, 44)

In [6]:
print "Step 2 : Manipulations for adjusting Charge-off - Pls examine for redundancy"

# creating a copy of the dataset
summ2 = all_balances[ all_balances['lusa_band'] != 'N/A']  # pd.read_csv('Performance_summary_6Jan2017.csv')

# adjustment to not include customer - month since origination combinations in the denominator that have not had the time to go 180+
summ2['CO_metrics_denominator_customers'] = np.where(summ2['exclude_co'] == 0, summ2['cust'], 0)

# Per accounting charge-off are recognized as such 180 days after non-payment; Defining customers on an open basis - probably redundant
summ2['CO_customers_this_mth'] = summ2['co_this_mth_accounting'] * summ2['cust']

# Same as above but recognizing Charge-offs at the first missing payment vs 180 days later - probabl redundant
summ2['CO_customers_this_mth_adj'] = np.where(summ2['exclude_co'] == 0,summ2['co_this_mth_accounting'] * summ2['cust'], 0)

# An adjustment to read Lines of Charged-off accounts - Probably redundant for this purpose
summ2['CO_Lines_adj'] = np.where(summ2['exclude_co'] == 0,summ2['co_this_mth_accounting'] * summ2['cust'] * summ2['max_credit_limit'], 0)

#adding because of adjustments needed
summ2['lusa_band1'] = summ2['lusa_band'].apply(lambda x: float(x.partition('(')[2].partition(',')[0]))


Step 2 : Manipulations for adjusting Charge-off - Pls examine for redundancy


/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.p

In [7]:
month_end_summ = pd.read_sql("""(WITH
        released_accounts AS (
          SELECT balances.account__id,
          min(balances.date::date) AS released_date, last_day(min(balances.date::date)) AS Cohort
          FROM origin.balances GROUP BY balances.account__id
          ),


        deltas AS (SELECT

                     deltas.account__id,
                     deltas.date,
                     greatest(asset__revolving + asset__late + asset__future_installments_issuer_interest + asset__installments_issuer,0) as sum_positive_mov,
                     (CASE when (asset__transitory_bank - liability__assignment_repurchase - profit_and_loss__assignment_loss) > 0 then 1 else 0 end) AS PMTS,
                     (liability__payable_brazil + liability__payable_foreign + liability__payable_installments) AS gmv,
                     (liability__payable_brazil + liability__payable_foreign) AS igmv,
                     (liability__payable_brazil) AS igmv_domestic,
                     (liability__payable_foreign) AS gmv_foreign,
                     (liability__payable_brazil + liability__payable_installments) AS gmv_domestic,
                     (profit_and_loss__installments_issuer_interest + profit_and_loss__installments_issuer_iof + profit_and_loss__revolving_interest) AS interest_revenue,
                     (profit_and_loss__installments_issuer_interest + profit_and_loss__installments_issuer_iof) AS interest_revenue_installments,
                     (profit_and_loss__revolving_interest) AS interest_revenue_normal,
                     --(profit_and_loss__forex_loss + profit_and_loss__forex_gain + profit_and_loss__fx_spread) AS fx_spread,
                     (profit_and_loss__fx_spread) AS fx_spread,
                     (profit_and_loss__identity_fraud_losses) AS identity_fraud_losses,
                     (profit_and_loss__fraud_losses) AS fraud_losses,
                     (profit_and_loss__credit_loss) AS credit_loss
                   FROM origin.deltas
                   ),


 Lusas AS
        (select account__id, lusa_v2_bin_25 as Lusa_band
            FROM dimension.prospect
        JOIN origin.account_key_integrity as k
            ON k.prospect__id = dimension.prospect.prospect__id
        --WHERE dimension.prospect.applied_at > '2017-03-01' 
    ),




                        Deltas_releases AS (SELECT

                          deltas.account__id, released_date,
                          deltas.date,
                          sum_positive_mov, pmts, gmv, igmv, igmv_domestic, gmv_foreign, gmv_domestic, interest_revenue, interest_revenue_installments,
                          interest_revenue_normal, fx_spread, identity_fraud_losses, fraud_losses, credit_loss

                                   FROM deltas
                                   JOIN released_accounts ON deltas.account__id = released_accounts.account__id
                                   JOIN Lusas ON deltas.account__id = Lusas.account__id
                                  --WHERE ((deltas.date - released_date) / 30) < ((CURRENT_DATE - released_date) / 30)
                                  ),





        Bals AS (SELECT
                      balances.account__id,
                      balances.date,
                      released_date,
                      (asset__closed + asset__settled_brazil + asset__settled_foreign + asset__unsettled) as adb_float_p,
                      (asset__settled_financed) as adb_float_set_fin,
                      (liability__payable_brazil + liability__payable_foreign) as adb_float_n,
                      (asset__revolving) as adb_revolving,
                      (asset__late) as adb_late,
                      (asset__future_installments_issuer_interest) as adb_issuer_interest,
                      (asset__installments_issuer) as adb_installments_issuer,
                      (asset__revolving + asset__late + asset__future_installments_issuer_interest + asset__installments_issuer) as adb,
                      Cohort,
                      Lusa_band

                      FROM origin.balances
                      JOIN released_accounts ON balances.account__id = released_accounts.account__id
                      JOIN Lusas ON balances.account__id = Lusas.account__id
                      --WHERE ((balances.date - released_date) / 30) < ((CURRENT_DATE - released_date) / 30)
                      ),




                    Deltas_Bals_releases AS( Select bals.account__id, bals.date, bals.released_date,
                                            sum_positive_mov, pmts, gmv, igmv, igmv_domestic, gmv_foreign, gmv_domestic, interest_revenue, interest_revenue_installments,
                                            interest_revenue_normal, fx_spread, identity_fraud_losses, fraud_losses, credit_loss, Cohort, Lusa_band,
                                            adb_float_p, adb_float_set_fin, adb_float_n, adb_revolving, adb_late, adb_issuer_interest, adb_installments_issuer,
                                            adb
                                            FROM Bals
                                            LEFT JOIN Deltas_releases ON Deltas_releases.account__id = Bals.account__id AND Deltas_releases.date = Bals.date
                                            ),


                    Perf_acc_lvl AS (Select deltas_Bals_releases.account__id,
                                               last_day(date) AS Mth,
                                               sum(sum_positive_mov) as New_balance_added,
                                               sum(pmts) as payments,
                                               sum(gmv) as purchase_f_gmv,
                                               sum(igmv) as purchase_f_igmv,
                                               sum(igmv_domestic) as purchase_f_igmv_domestic,
                                               sum(gmv_foreign) as purchase_f_gmv_foreign,
                                               sum(gmv_domestic) as purchase_f_gmv_domestic,
                                               sum(interest_revenue) as int_f_revenue,
                                               sum(interest_revenue_installments) as int_installments_f_revenue,
                                               sum(interest_revenue_normal) as int_f_revenue_normal,
                                               sum(fx_spread) as fx_spread_f,
                                               sum(identity_fraud_losses) as id_frd_f,
                                               sum(fraud_losses) as frd_f,
                                               sum(credit_loss) as credit_loss_f,
                                               avg(adb_float_p) as positive_float_adb,
                                               avg(adb_float_set_fin) as float_adb_fin,
                                               avg(adb_float_n) as negative_float_adb,
                                               avg(adb_revolving) as revolve_adb,
                                               avg(adb_late) as late_adb,
                                               avg(adb_issuer_interest) as issuer_int_adb,
                                               avg(adb_installments_issuer) as issuer_installments_adb,
                                               avg(adb) as adb_overall,
                                               1 as custs,
                                               Cohort, Lusa_band

                                               FROM Deltas_Bals_releases
                                               GROUP BY account__id, mth, Cohort, lusa_band
                                               ORDER BY account__id, mth, Cohort, lusa_band
                                               ),

                       Activity_flags AS(    SELECT Deltas_releases.account__id,
                                        (date_part('month', date)) + 100*date_part('year', date) AS Mth,
                                        (CASE when sum(gmv) > 0 then 1 else 0 end) as Purchase_active_Rate,
                                        (CASE when sum(interest_revenue) > 0 then 1 else 0 end) as Balance_active_Rate

                                            FROM Deltas_releases
                                            GROUP BY Account__id, mth
                                            Order BY account__id, mth
                                        ),

                       DQall AS(
                         select
                         date.date::date,
                         released_date,
                         delinquency_period_start,
                         date.date::date - released_date as days_since_release,
                         date.date::date - delinquency_period_start as days_delinquent,
                         (CASE when (1 + ((date.date::date - delinquency_period_start) / 30)) > 7 then 7 else (1 + ((date.date::date - delinquency_period_start) / 30)) end) as DQ_stat,
                         (date_part('month', date.date::date)) + 100*date_part('year', date.date::date) AS Mth,
                         delinquency_table.account__id
                         from origin.delinquency_table
                         join dimension.date on date.date::date between delinquency_period_start and coalesce(delinquency_period_end, CURRENT_DATE)
                         join released_accounts on delinquency_table.account__id = released_accounts.account__id

                         ),

                         DQS AS(Select DQall.account__id, DQall.mth, DQall.DQ_Stat, days_delinquent from DQall where DQall.date = last_day(DQall.date)),


                         Chargeoffmth AS(
                         Select DQall.account__id,
                         min(DQall.mth) as Chargeoff_mth,
                         min((DQall.delinquency_period_start - released_date) / 30) as chargeoff_bill_mth
                         from DQall
                         Where DQall.days_delinquent = 180
                         Group by account__id
                         Order by account__id
                         ),


                         outstanding_bals AS(
                         Select
                         balances.account__id,
                         (asset__revolving + asset__late + asset__installments_merchant + asset__installments_issuer + asset__future_installments_issuer_interest + asset__chargeback + asset__closed + asset__settled_brazil + asset__settled_financed + asset__settled_foreign + asset__future_installments_issuer_iof + asset__pre_loss) AS outstanding_balance,
                         asset__max_limit as max_credit_limit,
                         (asset__revolving + asset__late + asset__installments_merchant + asset__installments_issuer + asset__future_installments_issuer_interest + asset__chargeback + asset__closed + asset__settled_brazil + asset__settled_financed + asset__settled_foreign + asset__future_installments_issuer_iof + asset__pre_loss - profit_and_loss__credit_loss) AS chgoffamt_f,
                         (date_part('month', balances.date)) + 100*date_part('year', balances.date) AS Mth

                         from origin.balances
                         join released_accounts on balances.account__id = released_accounts.account__id
                         where balances.date = last_day(balances.date)
                         ),




                                          ACCT_mth_lvl_summary AS (SELECT Perf_acc_lvl.account__id, Perf_acc_lvl.mth, cohort, lusa_band, custs,
                                                            New_balance_added, payments, purchase_f_gmv, purchase_f_igmv, purchase_f_igmv_domestic, purchase_f_gmv_foreign, purchase_f_gmv_domestic,
                                                            int_f_revenue, int_installments_f_revenue, int_f_revenue_normal, fx_spread_f, id_frd_f, frd_f, credit_loss_f,
                                                            positive_float_adb, float_adb_fin, negative_float_adb, revolve_adb, late_adb, issuer_int_adb,
                                                            issuer_installments_adb, adb_overall, outstanding_balance, max_credit_limit, days_delinquent,

                                                            (CASE when date_part('year', cohort) < 2016 then 3.5 else 0.6 end) as custs_adj,

                                                            Activity_flags.Purchase_active_rate,
                                                            Activity_flags.Balance_active_rate,

                                                            Sum(custs) over (partition by Perf_acc_lvl.mth) as total_accts_in_mth,
                                                            Sum((CASE when date_part('year', cohort) < 2016 then 3.5 else 0.6 end)) over (partition by Perf_acc_lvl.mth) as total_adj_accts_in_mth,
                                                            Sum(New_balance_added) over (partition by Perf_acc_lvl.mth) as total_NB_in_mth,
                                                            Sum(payments) over (partition by Perf_acc_lvl.mth) as total_pmts_in_mth,
                                                            Sum(purchase_f_gmv) over (partition by Perf_acc_lvl.mth) as total_gmv_in_mth,
                                                            Sum(purchase_f_igmv) over (partition by Perf_acc_lvl.mth) as total_igmv_in_mth,
                                                            Sum(adb_overall) over (partition by Perf_acc_lvl.mth) as total_adb_in_mth,


                                                            (CASE   when DQS.DQ_Stat is null then 0
                                                                    else DQS.DQ_Stat end) as DQ_stat,

                                                            (CASE   when Chargeoffmth.Chargeoff_mth is null then 0
                                                                    else 1 end) as CO_stat,

                                                            (CASE   when Chargeoffmth.Chargeoff_bill_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_bill_mth > Perf_acc_lvl.mth then 0
                                                                    else  1 end) as CO_now,

                                                            (CASE   when Chargeoffmth.Chargeoff_bill_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_bill_mth = Perf_acc_lvl.mth then 1
                                                                    else  0 end) as CO_this_mth,

                                                            (CASE   when Chargeoffmth.Chargeoff_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_mth = Perf_acc_lvl.mth then 1
                                                                    else  0 end) as CO_this_mth_Accounting,

                                                            (CASE   when Chargeoffmth.Chargeoff_mth is null then 0
                                                                    when Chargeoffmth.Chargeoff_mth = Perf_acc_lvl.mth then chgoffamt_f
                                                                    else  0 end) as chargeoffamt_f

                                                            FROM Perf_acc_lvl
                                                            LEFT JOIN Activity_flags ON Perf_acc_lvl.account__id = Activity_flags.account__id AND Perf_acc_lvl.mth = Activity_flags.mth
                                                            LEFT JOIN DQS ON Perf_acc_lvl.account__id = DQS.account__id AND Perf_acc_lvl.mth = DQS.mth
                                                            LEFT JOIN Chargeoffmth ON Perf_acc_lvl.account__id = Chargeoffmth.account__id
                                                            LEFT JOIN outstanding_bals ON Perf_acc_lvl.account__id = outstanding_bals.account__id AND Perf_acc_lvl.mth = outstanding_bals.mth
                                                            ),



                     cohort_lvl_summary As (Select
                     mth as Date_eom,
                     cohort as chrt,


                     sum(custs)/(avg(total_accts_in_mth) + 0.00000000000000001) as Cust,
                     sum(custs_adj)/(avg(total_adj_accts_in_mth) + 0.00000000000000001) as Cust_adj,
                     sum(CASE when mth = cohort then 1
                         ELSE 0 END) AS cust_this_mth,
                     sum(custs) as released,
                     sum(New_balance_added)/(avg(total_NB_in_mth) + 0.00000000000000001) as New_balance_added,
                     sum(payments)/(avg(total_pmts_in_mth)+ 0.00000000000000001) as payments,
                     sum(purchase_f_gmv)/(avg(total_gmv_in_mth) + 0.00000000000000001) as purchase_f_gmv,
                     sum(purchase_f_igmv)/(avg(total_igmv_in_mth) + 0.00000000000000001) as purchase_f_igmv,
                     sum(adb_overall)/(avg(total_adb_in_mth) + 0.00000000000000001) as adb_overall


                     from ACCT_mth_lvl_summary
                     GROUP BY Date_eom, chrt
                     ORDER BY Date_eom, chrt
                      )


                     Select *,
                     datediff(month,chrt,date_eom) as mth,
                     (date_part('month', chrt) + 100*date_part('year', chrt)) AS Cohort
                     from cohort_lvl_summary
                     )
""",conn)

In [10]:
costs_proforma = pd.read_csv('Costs_inputs.csv')

In [11]:
print "Step 5 : Figuring out costs to be allocated to each cohort"
# Adjusting date from costs file to reflect a YYYYMM format for merging with SQL pulled performance
costs_proforma['date'] = (100 * pd.to_datetime(costs_proforma['Date']).dt.year) + (pd.to_datetime(costs_proforma['Date']).dt.month)

# Adjusting date from costs file to reflect a YYYYMM format for merging with SQL pulled performance
month_end_summ['date'] = (100 * pd.to_datetime(month_end_summ['date_eom']).dt.year) + (pd.to_datetime(month_end_summ['date_eom']).dt.month)

# Merging the two files for allocation
costs_merged = pd.merge(month_end_summ, costs_proforma, on='date')

# Conductor/Authorizer costs - Monthly Costs from VAMP allocated on the basis of # of customers booked in each cohort
costs_merged['Authorizer_Costs'] = -1000 * costs_merged['Authorizer_Costs'] * costs_merged['cust'] / costs_merged['released']

# Boleto costs - Allocated to each cohort based on payment made from each cohort
costs_merged['Partner Bank Bill Charging Costs'] = -1000 * costs_merged['Partner_Bank_Bill_Charging_costs'] * costs_merged['payments'] / costs_merged['released']

# Technology and Communication costs - Allocated based on number of customers in each cohort; Assumed to scale up and down with accounts in each cohort
costs_merged['Technology & Communication'] = -1000 * costs_merged['Tech_n_Comm_costs'] * costs_merged['cust'] / \
                                             costs_merged['released']

# Acquisiton costs incl. Printing and Shipping plastic + cost incurred to pull from Bureau - Allocated only to customers originated in the same cohort as the month of incurring cost
costs_merged['Printing Shipping and Data Costs'] = -1000 * costs_merged['Printing_Shipping_Data_costs'] * \
                                                   costs_merged['cust_this_mth'] / (
                                                   costs_merged['released'] * costs_merged['released'])

# Marketing costs - Allocated only to customers originated in the same cohort as the month of incurring cost
costs_merged['Marketing Expenses'] = -1000 * costs_merged['Marketing_expenses'] * costs_merged['cust_this_mth'] / (
costs_merged['released'] * costs_merged['released'])

# Costs incurred due to personnel engaged in account opening procedures - allocated to the cohort of the month
costs_merged['Personnel + G&A (Cust Acquisition)'] = -1000 * costs_merged['Customer_Experience_Acq_costs'] * \
                                                     costs_merged['cust_this_mth'] / (
                                                     costs_merged['released'] * costs_merged['released'])

# Servicing costs - allocated based on accounts in each cohort
costs_merged['Personnel + G&A (Cust Service)'] = -1000 * costs_merged['Customer_Mgmt_Costs'] * costs_merged[
    'cust'] / costs_merged['released']

# Costs paid to Mastercard linked to spending - Allocated based on transaction volume in each cohort
costs_merged['MC_spend_linked_cost_per_igmv'] = -1000 * costs_merged['MasterCard_spend_linked_costs'] * costs_merged['purchase_f_igmv'] / costs_merged['released']

# Costs paid to Mastercard linked to spending - Allocated based on transaction volume in each cohort
costs_merged['Bracce_cost_per_released'] = -1000 * costs_merged['Partner_Bank_Credit_Issuance_Costs'] * costs_merged['new_balance_added'] / costs_merged['released']

# Costs paid to Mastercard linked to account servicing - Allocated based on customers in each cohort; Customers are adjusted based on whether they are Platinum or GOld
costs_merged['MC_account_linked_cost_per_released'] = -1000 * costs_merged['MasterCard_card_linked_costs'] * costs_merged['cust_adj'] / costs_merged['released']

# redundant
costs_merged['Platinum_MC_account_linked_cost_per_released'] = 0

# redundant
costs_merged['Gold_MC_account_linked_cost_per_released'] = 0

# Retaining only those columns which will be used in the future
costs_merged_final = costs_merged[['cohort', 'mth', 'Authorizer_Costs', 'Partner Bank Bill Charging Costs', 'Technology & Communication',
     'Printing Shipping and Data Costs', 'Marketing Expenses', 'Personnel + G&A (Cust Acquisition)',
     'Personnel + G&A (Cust Service)', 'MC_spend_linked_cost_per_igmv', 'Bracce_cost_per_released',
     'MC_account_linked_cost_per_released', 'Gold_MC_account_linked_cost_per_released',
     'Platinum_MC_account_linked_cost_per_released']]

Step 5 : Figuring out costs to be allocated to each cohort


In [12]:
print "Step 6 : Bringing customer performance together with allocated costs from VAMP"
    # Merging to bring VAMP costs at cohort mth level on a per released basis to 30 day interval of each cohort
summ3 = summ2.merge(costs_merged_final, on=['cohort', 'mth'], how='left')

# Defining late bucket balances to calculate Provisions later
summ3['Current_Balance_Transactor'] = np.where(summ3['dq_stat'] == 0,
                                               summ3['outstanding_balance'] - summ3['adb_overall'], 0)
summ3['Current_Balance_Revolver'] = np.where(summ3['dq_stat'] == 0, summ3['adb_overall'], 0)
summ3['0-30 Balance'] = np.where(summ3['dq_stat'] == 1, summ3['outstanding_balance'], 0)
summ3['30-60 Balance'] = np.where(summ3['dq_stat'] == 2, summ3['outstanding_balance'], 0)
summ3['60-90 Balance'] = np.where(summ3['dq_stat'] == 3, summ3['outstanding_balance'], 0)
summ3['90-120 Balance'] = np.where(summ3['dq_stat'] == 4, summ3['outstanding_balance'], 0)
summ3['120-150 Balance'] = np.where(summ3['dq_stat'] == 5, summ3['outstanding_balance'], 0)
summ3['150-180 Balance'] = np.where(summ3['dq_stat'] == 6, summ3['outstanding_balance'], 0)

# Bringing costs from VAMP into customer performance
summ3['Plat_MC_cost'] = summ3['Platinum_MC_account_linked_cost_per_released'] * summ3['cust']
summ3['Gold_MC_cost'] = summ3['Gold_MC_account_linked_cost_per_released'] * summ3['cust']
summ3['MC_cost'] = summ3['MC_account_linked_cost_per_released'] * summ3['cust']
summ3['Conductor_cost'] = summ3['Authorizer_Costs'] * summ3['cust']
summ3['Bill_costs'] = summ3['Partner Bank Bill Charging Costs'] * summ3['cust']
summ3['MC_spend_linked_costs'] = summ3['MC_spend_linked_cost_per_igmv'] * summ3['cust']
summ3['Bracce_costs'] = summ3['Bracce_cost_per_released'] * summ3['cust']
summ3['Tech_n_Comm_costs'] = summ3['Technology & Communication'] * summ3['cust']
summ3['Cust_serv_n_Collections_costs'] = summ3['Personnel + G&A (Cust Service)'] * summ3['cust']
summ3['Print_ship_data_costs'] = (summ3['Printing Shipping and Data Costs']) * summ3['cust']
summ3['Mktg_costs'] = summ3['Marketing Expenses'] * summ3['cust']
summ3['Acq_Personnel_costs'] = summ3['Personnel + G&A (Cust Acquisition)'] * summ3['cust']

summ3['acq_cost'] = summ3['Print_ship_data_costs'] + summ3['Mktg_costs'] + summ3['Acq_Personnel_costs']
summ3['Platinum_custs'] = np.where(summ3['cohort'] <= 201512, summ3['cust'], 0)
summ3['cohort'] = summ3['cohort'].astype(int)


Step 6 : Bringing customer performance together with allocated costs from VAMP


In [13]:
print "Step 7 : Reading a static file of provision rates for different rate buckets varying by Lusa and months since origination"



# This is a static file create in Dec'16; May need updates and improvements
provs = pd.read_csv('Provision_rate_to_use_Lusav2.csv')


Step 7 : Reading a static file of provision rates for different rate buckets varying by Lusa and months since origination


In [14]:
provision_table[provision_table.days_late == 0]

,days_late,lusa_v2_band,mth,lusa_v2_bin_25,provision_rate,provision_rate_mult
0,0,400.0,0,"(325, 350]",0.004523,4522.877395
1,0,400.0,1,"(325, 350]",0.026222,26222.361236
2,0,400.0,2,"(325, 350]",0.028326,28325.937550
3,0,400.0,3,"(325, 350]",0.028620,28620.073167
4,0,400.0,4,"(325, 350]",0.025917,25916.546384
5,0,400.0,5,"(325, 350]",0.025779,25779.160140
6,0,400.0,6,"(325, 350]",0.022694,22693.504241
7,0,400.0,7,"(325, 350]",0.022163,22163.034900
8,0,400.0,8,"(325, 350]",0.022237,22236.520304
9,0,400.0,9,"(325, 350]",0.020758,20758.376639


In [15]:
print "Step 8 : Merging performance with provision table varying across time "

# Merging Provisions rates with Performance summary - Provision rates vary by buckets and by lusa and mth
summ4 = summ3.merge(provision_table[provision_table.days_late==0], how='left', left_on=['lusa_band', 'mth'], right_on=['lusa_v2_bin_25','mth'])

# Condition to revert to a flat rate (implied from VAMP) when mth >= 12 as data to estimate may be thin there
summ4['prov_for_current'] = np.where((summ4['mth'] < 13), summ4['provision_rate'] *  (summ4['Current_Balance_Transactor'] + summ4['Current_Balance_Revolver']), 0.0126 * (summ4['Current_Balance_Transactor'] + summ4['Current_Balance_Revolver']))
summ4['prov_for_0to30'] = np.where((summ4['mth'] < 13), np.where(summ4['dq_stat'] == 1, summ4['provision_balance'], 0), 0.2690 * summ4['0-30 Balance'])
summ4['prov_for_30to60'] = np.where((summ4['mth'] < 13), np.where(summ4['dq_stat'] == 2, summ4['provision_balance'], 0), 0.6283 * summ4['30-60 Balance'])
summ4['prov_for_60to90'] = np.where((summ4['mth'] < 13), np.where(summ4['dq_stat'] == 3, summ4['provision_balance'], 0), 0.8629 * summ4['60-90 Balance'])
summ4['prov_for_90to120'] = np.where((summ4['mth'] < 13),np.where(summ4['dq_stat']== 4, summ4['provision_balance'], 0), 0.9394 * summ4['90-120 Balance'])
summ4['prov_for_120to150'] = np.where((summ4['mth'] < 13), np.where(summ4['dq_stat'] == 5, summ4['provision_balance'], 0), 0.9787 * summ4['120-150 Balance'])
summ4['prov_for_150to180'] = 1 * summ4['150-180 Balance']

Step 8 : Merging performance with provision table varying across time 


In [16]:
 print "Step 9 : Generating a list of cohorts for which to run monitoring"

# Set of manipulations to get to a list of monthly cohorts and then some related aggregations
cohort_list = summ4[(summ4.exclude_censor == 0) & (summ4.cohort >= 201410)].cohort.sort_values(ascending=True).drop_duplicates().reset_index().reset_index()
cohort_list.level_0.max()
cohort_list['Last 3 months'] = np.where(cohort_list.level_0 <= cohort_list.level_0.max() - 3, 0, 1)
cohort_list['Last 6 months'] = np.where(cohort_list.level_0 <= cohort_list.level_0.max() - 6, 0, 1)
cohort_list['Last 12 months'] = np.where(cohort_list.level_0 <= cohort_list.level_0.max() - 12, 0, 1)
cohort_list['All'] = 1
cohort_list['code'] = cohort_list['All'] + cohort_list['Last 3 months'] + cohort_list['Last 6 months'] + cohort_list['Last 12 months']

# Set of manipulations to get to a list of quarterly cohorts
map_for_qtr_vintages_included = {1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 2, 7: 3, 8: 3, 9: 3, 10: 4, 11: 4, 12: 4}

cohort_list['month'] = cohort_list['cohort'] - (100 * (cohort_list['cohort'] // 100))
cohort_list['year'] = cohort_list['cohort'] // 100
cohort_list['qtr'] = cohort_list.apply(lambda row: '{year}Q{quarter}'.format(year=row['cohort'] // 100,
                                                                             quarter=int(((row['cohort'] - (100 * (
                                                                             row[
                                                                                 'cohort'] // 100))) * .99) // 3 + 1)),
                                       axis=1)

cohort_list2 = cohort_list.groupby(['qtr']).count()[['code']].reset_index().reset_index()

cohort_list2.rename(columns={'index': 'qtr_index'}, inplace=True)

cohort_list3 = cohort_list.merge(cohort_list2[['qtr_index', 'qtr']], on=['qtr'])
cohort_list3

Step 9 : Generating a list of cohorts for which to run monitoring


,level_0,index,cohort,Last 3 months,Last 6 months,Last 12 months,All,code,month,year,qtr,qtr_index
0,0,0,201410,0,0,0,1,1,10,2014,2014Q4,0
1,1,160840,201411,0,0,0,1,1,11,2014,2014Q4,0
2,2,279542,201412,0,0,0,1,1,12,2014,2014Q4,0
3,3,30723,201501,0,0,0,1,1,1,2015,2015Q1,1
4,4,2597,201502,0,0,0,1,1,2,2015,2015Q1,1
5,5,124012,201503,0,0,0,1,1,3,2015,2015Q1,1
6,6,93502,201504,0,0,0,1,1,4,2015,2015Q2,2
7,7,131963,201505,0,0,0,1,1,5,2015,2015Q2,2
8,8,129774,201506,0,0,0,1,1,6,2015,2015Q2,2
9,9,165640,201507,0,0,0,1,1,7,2015,2015Q3,3


In [37]:
# reading input file
print "Step 10 : Generating a list of cohorts for which to run monitoring"
ledgers = pd.read_csv('ledgers_concat_stable_version_july_4th.csv')

# Manipulations for basic use and merges in the future
ledgers['Metric'] = ledgers['Unnamed: 1'] + '_' + ledgers['Unnamed: 2'] + '_' + ledgers['Unnamed: 3']
ledgers

ledgers1 = ledgers.set_index(['Metric', 'Unnamed: 0']).drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'],1).T.reset_index().reset_index()
ledgers2 = ledgers1.drop(['index', 'level_0'], 1).stack().reset_index()
ledgers2['Type'] = 'Forecast'
ledgers2['mth'] = ledgers2['level_0'] - 1
dct = {'(000, 025]' : '(0, 25]','(025, 050]' : '(25, 50]','(050, 075]' : '(50, 75]','(075, 100]' : '(75, 100]','(100, 125]' : '(100, 125]','(125, 150]' : '(125, 150]','(150, 175]' : '(150, 175]','(175, 200]' : '(175, 200]','(200, 225]' : '(200, 225]','(225, 250]' : '(225, 250]','(250, 275]' : '(250, 275]','(275, 300]' : '(275, 300]','(300, 325]' : '(300, 325]','(325, 350]' : '(325, 350]','(350, 375]' : '(350, 375]','(375, 400]' : '(375, 400]','(400, 425]' : '(400, 425]','(425, 450]' : '(425, 450]','(450, 475]' : '(450, 475]','(475, 500]' : '(475, 500]','(500, 525]' : '(500, 525]','(525, 550]' : '(525, 550]','(550, 575]' : '(550, 575]','(575, 600]' : '(575, 600]','(600, 625]' : '(600, 625]','(625, 650]' : '(625, 650]','(650, 675]' : '(650, 675]','(675, 700]' : '(675, 700]','(700, 725]' : '(700, 725]','(725, 750]' : '(725, 750]','(750, 775]' : '(750, 775]','(775, 800]' : '(775, 800]','(800, 825]' : '(800, 825]','(825, 850]' : '(825, 850]','(850, 875]' : '(850, 875]','(875, 900]' : '(875, 900]','(900, 925]' : '(900, 925]','(925, 950]' : '(925, 950]','(950, 975]' : '(950, 975]','(975, 1000]' : '(975, 1000]'}
ledgers2['lusa_band'] = map(dct.get, ledgers2['Unnamed: 0'])

# Adjusting Charge-off to make it comparable with Cohort Monitoring
ledgers2['CO_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['accs_asset_released_activated_charged_off_recognized'].diff()
ledgers2['CO'] = np.where(ledgers2['CO_delta'].isnull(),ledgers2['accs_asset_released_activated_charged_off_recognized'], ledgers2['CO_delta'])
ledgers2['CO_shifted'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['CO'].shift(-6)

ledgers2['losses_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_recognized_losses'].diff()
ledgers2['losses'] = np.where(ledgers2['losses_delta'].isnull(), ledgers2['main_pandl_recognized_losses'],ledgers2['losses_delta'])
# ledgers2['losses'] = ledgers2.sort_values(['Unnamed: 0' , 'mth']).groupby(['Unnamed: 0'])['losses_x'].shift(-1)
ledgers2['losses_shifted'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['losses'].shift(-6)

# Manipulations to turn losses in Atom into a Perfect Provisioning structure
ledgers2['losses_shifted_7'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['losses'].shift(-7).fillna(0)
ledgers2['losses_shifted_6'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['losses'].shift(-6).fillna(0)
ledgers2['losses_shifted_5'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['losses'].shift(-5).fillna(0)
ledgers2['losses_shifted_4'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['losses'].shift(-4).fillna(0)
ledgers2['losses_shifted_3'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['losses'].shift(-3).fillna(0)
ledgers2['losses_shifted_2'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['losses'].shift(-2).fillna(0)
ledgers2['losses_shifted_1'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['losses'].shift(-1).fillna(0)

ledgers2['Perfect_provision'] = (ledgers2['losses_shifted_7'] + ledgers2['losses_shifted_6'] + ledgers2['losses_shifted_5'] + ledgers2['losses_shifted_4'] + ledgers2['losses_shifted_3'] + ledgers2['losses_shifted_2'] + ledgers2['losses_shifted_1'])
ledgers2['Change_in_Provision'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['Perfect_provision'].diff()
ledgers2['Change_in_Provision'] = np.where(ledgers2.mth == 0, ledgers2['Perfect_provision'],ledgers2['Change_in_Provision'])

# Implying Fx spread; Ledger is cumulative - This step decouples it
ledgers2['fx_spread_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_fx_spread_revenues'].diff()
ledgers2['fx_spread'] = np.where(ledgers2['fx_spread_delta'].isnull(), ledgers2['main_pandl_fx_spread_revenues'],ledgers2['fx_spread_delta'])

ledgers2['fx_spread_taxes_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_fx_spread_rev_taxes'].diff()
ledgers2['fx_spread_taxes'] = np.where(ledgers2['fx_spread_taxes_delta'].isnull(),ledgers2['main_pandl_fx_spread_rev_taxes'],ledgers2['fx_spread_taxes_delta'])

# Implying Transaction fraud; Ledger is cumulative - This step decouples it
ledgers2['txn_frd_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_txn_fraud_costs'].diff()
ledgers2['txn_frd'] = np.where(ledgers2['txn_frd_delta'].isnull(), ledgers2['main_pandl_txn_fraud_costs'],ledgers2['txn_frd_delta'])

# Implying Interest Expense and revenues from Atom; Ledger is cumulative - This step decouples it
if 'main_pandl_interest_expense' in ledgers2.columns:

    ledgers2['interest_expense_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_interest_expense'].diff()
    ledgers2['interest_expense'] = np.where(ledgers2['interest_expense_delta'].isnull(),ledgers2['main_pandl_interest_expense'], ledgers2['interest_expense_delta'])

else:
    ledgers2['interest_expense_delta'] = 0
    ledgers2['interest_expense'] = 0
    

if 'main_pandl_interest_rev_taxes' in ledgers2.columns:
    ledgers2['interest_taxes_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_interest_rev_taxes'].diff()
    ledgers2['interest_taxes'] = np.where(ledgers2['interest_taxes_delta'].isnull(),ledgers2['main_pandl_interest_rev_taxes'], ledgers2['interest_taxes_delta'])
else:   
    ledgers2['interest_taxes_delta'] = 0
    ledgers2['interest_taxes'] = 0

if 'main_pandl_interest_revenues' in ledgers2.columns:
    ledgers2['interest_revenues_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_interest_revenues'].diff()
    ledgers2['interest_revenues'] = np.where(ledgers2['interest_revenues_delta'].isnull(),ledgers2['main_pandl_interest_revenues'],ledgers2['interest_revenues_delta'])
else:
    ledgers2['interest_revenues_delta']=0
    ledgers2['interest_revenues']=0
    

    
if 'main_pandl_late_fee_tax' in ledgers2.columns:   
    ledgers2['lfee_taxes_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_late_fee_tax'].diff()
    ledgers2['lfee_taxes'] = np.where(ledgers2['lfee_taxes_delta'].isnull(),ledgers2['main_pandl_late_fee_tax'], ledgers2['lfee_taxes_delta'])
else:
    ledgers2['lfee_taxes_delta'] = 0
    ledgers2['lfee_taxes'] = 0
    

if 'main_pandl_late_fee_revenue' in ledgers2.columns:
    ledgers2['lfee_revenues_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_late_fee_revenue'].diff()
    ledgers2['lfee_revenues'] = np.where(ledgers2['lfee_revenues_delta'].isnull(),ledgers2['main_pandl_late_fee_revenue'],ledgers2['lfee_revenues_delta'])
else:
    ledgers2['lfee_revenues_delta'] = 0
    ledgers2['lfee_revenues'] = 0
    
if 'main_pandl_credit_loss_recovery' in ledgers2.columns:
    ledgers2['recovery_delta'] = ledgers2.sort_values(['Unnamed: 0', 'mth']).groupby(['Unnamed: 0'])['main_pandl_credit_loss_recovery'].diff()
    ledgers2['credit_recoveries'] = np.where(ledgers2['recovery_delta'].isnull(), ledgers2['main_pandl_credit_loss_recovery'],ledgers2['recovery_delta'])
else:
    ledgers2['recovery_delta'] = 0
    ledgers2['credit_recoveries'] = 0
    
    

Step 10 : Generating a list of cohorts for which to run monitoring


/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



## Core Functions

In [18]:
def cm(ca, summ3, Lusa_cutoff=0, At_scale_costs="No", Platinum_Gold="None"):

    # Excluding pre 201410 cohorts ---- ca is the list of cohorts to run   ---- excluding mths with partial data --- summarizing all data created upto now for each cohort
    Cohort_metrics = summ3[(summ3.cohort >= 201410) & (summ3.cohort.isin(ca)) & (summ3.exclude_censor == 0) & (
    summ3.lusa_band1 >= Lusa_cutoff)].groupby(['mth']).sum()[
        ['Platinum_custs', 'new_balance_added', 'fx_spread_f', 'int_f_revenue', 'acq_cost', 'purchase_f_gmv','avg_outstanding_bal','outstanding_balance',
         'purchase_f_igmv', 'chargeoffamt_f', 'credit_loss_f','credit_loss_recovery_f', 'max_credit_limit', 'adb_overall', 'revolve_adb',
         'late_adb', 'issuer_int_adb','issuer_installments_adb','adb_other','frd_f', 'id_frd_f',
         'CO_Lines_adj', 'MC_spend_linked_costs', 'CO_customers_this_mth_adj', 'cust', 'late_fee_f', 'rewards_fee_f',
         'CO_metrics_denominator_customers', 'purchase_f_igmv_domestic', 'purchase_f_gmv_domestic',
         'purchase_f_gmv_foreign', '0-30 Balance', '30-60 Balance', '60-90 Balance', '90-120 Balance',
         '120-150 Balance', '150-180 Balance', 'Current_Balance_Transactor', 'Current_Balance_Revolver', 'Plat_MC_cost',
         'Gold_MC_cost', 'Conductor_cost', 'Bill_costs', 'Tech_n_Comm_costs', 'Cust_serv_n_Collections_costs',
         'Print_ship_data_costs', 'Acq_Personnel_costs', 'Mktg_costs', 'MC_cost', 'Bracce_costs', 'prov_for_current',
         'prov_for_0to30', 'prov_for_30to60', 'prov_for_60to90', 'prov_for_90to120', 'prov_for_120to150']]

    # GMV per released ----- purchase_f_gmv is difference in (liability__payable_brazil + liability__payable_foreign + liability__payable_installments)
    Cohort_metrics['01 Purchase_gmv_per_released'] = Cohort_metrics['purchase_f_gmv'] / Cohort_metrics['cust']

    # GMV per released ----- purchase_f_igmv is difference in (liability__payable_brazil + liability__payable_foreign)
    Cohort_metrics['02 Purchase_igmv_per_released'] = Cohort_metrics['purchase_f_igmv'] / Cohort_metrics['cust']

    # redundant
    Cohort_metrics['Platinum_per_released'] = Cohort_metrics['Platinum_custs'] / Cohort_metrics['cust']

    # Most below are reflective of current assumptions
    Cohort_metrics['03 Purchase_igmv_domestic_per_released'] = Cohort_metrics['purchase_f_igmv_domestic'] / Cohort_metrics['cust']
    Cohort_metrics['04 Platinum_Gross_domestic_Interchange_revenue_per_released'] = PLATINUM_DOMESTIC_INTERCHANGE_RATE * Cohort_metrics['03 Purchase_igmv_domestic_per_released']
    Cohort_metrics['05 Platinum_Taxes_domestic_Interchange_revenue_per_released'] = OPERATING_REVENUES_TAXES * Cohort_metrics['04 Platinum_Gross_domestic_Interchange_revenue_per_released']
    Cohort_metrics['06 Platinum_Net_domestic_Interchange_revenue_per_released'] = Cohort_metrics['04 Platinum_Gross_domestic_Interchange_revenue_per_released'] - Cohort_metrics['05 Platinum_Taxes_domestic_Interchange_revenue_per_released']

    Cohort_metrics['07 Gold_Gross_domestic_Interchange_revenue_per_released'] = GOLD_DOMESTIC_INTERCHANGE_RATE * Cohort_metrics['03 Purchase_igmv_domestic_per_released']
    Cohort_metrics['08 Gold_Taxes_domestic_Interchange_revenue_per_released'] = OPERATING_REVENUES_TAXES * Cohort_metrics['07 Gold_Gross_domestic_Interchange_revenue_per_released']
    Cohort_metrics['09 Gold_Net_domestic_Interchange_revenue_per_released'] = Cohort_metrics['07 Gold_Gross_domestic_Interchange_revenue_per_released'] - Cohort_metrics['08 Gold_Taxes_domestic_Interchange_revenue_per_released']

    # Combining for platinum and gold customers
    Cohort_metrics['07a Gross_domestic_Interchange_revenue_per_released'] = np.where(Platinum_Gold == "Platinum",Cohort_metrics['04 Platinum_Gross_domestic_Interchange_revenue_per_released'],np.where(Platinum_Gold == "Gold",Cohort_metrics['07 Gold_Gross_domestic_Interchange_revenue_per_released'],(Cohort_metrics['Platinum_per_released'] *Cohort_metrics['04 Platinum_Gross_domestic_Interchange_revenue_per_released']) + ((1 - Cohort_metrics['Platinum_per_released']) *Cohort_metrics['07 Gold_Gross_domestic_Interchange_revenue_per_released'])))
    Cohort_metrics['08a Taxes_domestic_Interchange_revenue_per_released'] = OPERATING_REVENUES_TAXES * Cohort_metrics['07a Gross_domestic_Interchange_revenue_per_released']
    Cohort_metrics['09a Net_domestic_Interchange_revenue_per_released'] = Cohort_metrics['07a Gross_domestic_Interchange_revenue_per_released'] - Cohort_metrics['08a Taxes_domestic_Interchange_revenue_per_released']

    # Most below are reflective of current assumptions
    Cohort_metrics['10 Purchase_igmv_international_per_released'] = Cohort_metrics['purchase_f_gmv_foreign'] / Cohort_metrics['cust']
    Cohort_metrics['11 Purchase_igmv_international_interchange_per_released'] = FOREIGN_INTERCHANGE_RATE * Cohort_metrics['10 Purchase_igmv_international_per_released']
    Cohort_metrics['12 Purchase_igmv_international_Taxes_interchange_per_released'] = OPERATING_REVENUES_TAXES * Cohort_metrics['11 Purchase_igmv_international_interchange_per_released']
    Cohort_metrics['13 Net_international_interchange_per_released'] = Cohort_metrics['11 Purchase_igmv_international_interchange_per_released'] - Cohort_metrics['12 Purchase_igmv_international_Taxes_interchange_per_released']

    # fx_spread_f = difference from one month to the other on (profit_and_loss__forex_loss + profit_and_loss__forex_gain + profit_and_loss__fx_spread)
    Cohort_metrics['14 fx_spread_per_released'] = Cohort_metrics['fx_spread_f'] / Cohort_metrics['cust']
    Cohort_metrics['15 fx_spread_taxes_per_released'] = FINANCIAL_REVENUES_TAXES * Cohort_metrics['14 fx_spread_per_released']
    Cohort_metrics['16 net_fx_spread_per_released'] = Cohort_metrics['14 fx_spread_per_released'] - Cohort_metrics['15 fx_spread_taxes_per_released']
    Cohort_metrics['17 net_fx_rate'] = Cohort_metrics['16 net_fx_spread_per_released'] / Cohort_metrics['02 Purchase_igmv_per_released']

    # 18 and 19 are redundant; Rest coming from VAMP
    Cohort_metrics['18 MC_spend_linked_domestic_per_released'] = MC_SPEND_LINKED_DOMESTIC_RATE * Cohort_metrics['03 Purchase_igmv_domestic_per_released']
    Cohort_metrics['19 MC_spend_linked_international_per_released'] = MC_SPEND_LINKED_FOREIGN_RATE * Cohort_metrics['10 Purchase_igmv_international_per_released']
    Cohort_metrics['20 MC_spend_linked_costs_per_released'] = (Cohort_metrics['MC_spend_linked_costs']) / Cohort_metrics['cust']
    Cohort_metrics['21 MC_spend_linked_costs_rate'] = Cohort_metrics['20 MC_spend_linked_costs_per_released'] / Cohort_metrics['02 Purchase_igmv_per_released']

    # 22 and 23 redundant -----> Net non-interest income = Net Domestic Intchng + Net International Int'chng + Net FX spread - Net Mastercard Spend Linked costs
    Cohort_metrics['22 Platinum_Net_non_interest_income_per_released'] = Cohort_metrics['06 Platinum_Net_domestic_Interchange_revenue_per_released'] + Cohort_metrics['13 Net_international_interchange_per_released'] + Cohort_metrics['16 net_fx_spread_per_released'] - Cohort_metrics['20 MC_spend_linked_costs_per_released']
    Cohort_metrics['23 Gold_Net_non_interest_income_per_released'] = Cohort_metrics['09 Gold_Net_domestic_Interchange_revenue_per_released'] + Cohort_metrics['13 Net_international_interchange_per_released'] + Cohort_metrics['16 net_fx_spread_per_released'] - Cohort_metrics['20 MC_spend_linked_costs_per_released']
    Cohort_metrics['23a Net_non_interest_income_per_released'] = Cohort_metrics['09a Net_domestic_Interchange_revenue_per_released'] + Cohort_metrics['13 Net_international_interchange_per_released'] + Cohort_metrics['16 net_fx_spread_per_released'] - Cohort_metrics['20 MC_spend_linked_costs_per_released']

    # Only info -- redundant
    Cohort_metrics['24 Platinum_Interchange_rate'] = (Cohort_metrics['06 Platinum_Net_domestic_Interchange_revenue_per_released'] +Cohort_metrics['13 Net_international_interchange_per_released']) / Cohort_metrics['02 Purchase_igmv_per_released']
    Cohort_metrics['25 Gold_Interchange_rate'] = (Cohort_metrics['09 Gold_Net_domestic_Interchange_revenue_per_released'] +Cohort_metrics['13 Net_international_interchange_per_released']) / Cohort_metrics['02 Purchase_igmv_per_released']
    Cohort_metrics['25a Interchange_rate'] = (Cohort_metrics['09a Net_domestic_Interchange_revenue_per_released'] +Cohort_metrics['13 Net_international_interchange_per_released']) / Cohort_metrics['02 Purchase_igmv_per_released']

    # 26 and 27 redundant ----- Real answer coming from VAMP
    Cohort_metrics['26 Platinum_MC_account_linked_cost_per_released'] = (Cohort_metrics['Plat_MC_cost']) / Cohort_metrics['cust']
    Cohort_metrics['27 Gold_MC_account_linked_cost_per_released'] = (Cohort_metrics['Gold_MC_cost']) / Cohort_metrics['cust']
    Cohort_metrics['27a MC_account_linked_cost_per_released'] = np.where(Platinum_Gold == "Platinum", 4.2,np.where(Platinum_Gold == "Gold", 0.6,(Cohort_metrics['MC_cost']) /Cohort_metrics['cust']))

    # coming from VAMp
    Cohort_metrics['28 Conductor_cost_per_released'] = np.where(At_scale_costs == "Yes", CONDUCTOR_COST_PER_RELEASED_SCALE,(Cohort_metrics['Conductor_cost']) / Cohort_metrics['cust'])
    Cohort_metrics['29 Partner_Bank_Bill_charging_cost_per_released'] = Cohort_metrics['Bill_costs'] / Cohort_metrics['cust']  # 1.24 per payment

    # One of them is not used
    Cohort_metrics['30 Bracce_cost_per_released'] = Cohort_metrics['Bracce_costs'] / Cohort_metrics['cust']

    #TODO IS THIS A TYPO OR ARE WE REFERRING TO BRACCE COSTS AGAIN?
    Cohort_metrics['31 Sorocred_cost_per_released'] = Cohort_metrics['Bracce_costs'] / Cohort_metrics['cust']

    # Coming from VAMP
    Cohort_metrics['32 Reissunace_cost_per_released'] = REISSUANCE_COST_PER_RELEASED
    Cohort_metrics['33 Collections_cost_per_released'] = COLLECTIONS_COST_PER_RELEASED
    Cohort_metrics['34 Cust_Mgmt_cost_per_released'] = np.where(At_scale_costs == "Yes", CUST_MGMT_COST_PER_RELEASED,Cohort_metrics['Cust_serv_n_Collections_costs'] /Cohort_metrics['cust'])
    Cohort_metrics['35 Tech_Comm_cost_per_released'] = np.where(At_scale_costs == "Yes", TECH_COMM_COST_PER_RELEASED,Cohort_metrics['Tech_n_Comm_costs'] / Cohort_metrics['cust'])

    # 36 and 37 redundant; Non Interest Expense = MC account linked + Boleto + Conductor + Cust Mgmt + Services + Tech n Comm + Bracce/Sorocred
    Cohort_metrics['36 Platinum_Non_Interest_Expense_per_released'] = (Cohort_metrics['26 Platinum_MC_account_linked_cost_per_released'] + Cohort_metrics['35 Tech_Comm_cost_per_released'] + Cohort_metrics['34 Cust_Mgmt_cost_per_released'] + Cohort_metrics['33 Collections_cost_per_released'] + Cohort_metrics['32 Reissunace_cost_per_released'] + Cohort_metrics['29 Partner_Bank_Bill_charging_cost_per_released'] + Cohort_metrics['28 Conductor_cost_per_released'] +Cohort_metrics['31 Sorocred_cost_per_released'])
    Cohort_metrics['37 Gold_Non_Interest_Expense_per_released'] = (Cohort_metrics['27 Gold_MC_account_linked_cost_per_released'] + Cohort_metrics['35 Tech_Comm_cost_per_released'] +Cohort_metrics['34 Cust_Mgmt_cost_per_released'] + Cohort_metrics['33 Collections_cost_per_released'] +Cohort_metrics['32 Reissunace_cost_per_released'] + Cohort_metrics['29 Partner_Bank_Bill_charging_cost_per_released'] + Cohort_metrics['28 Conductor_cost_per_released'] +Cohort_metrics['31 Sorocred_cost_per_released'])
    Cohort_metrics['37a Non_Interest_Expense_per_released'] = (Cohort_metrics['27a MC_account_linked_cost_per_released'] + Cohort_metrics['35 Tech_Comm_cost_per_released'] +Cohort_metrics['34 Cust_Mgmt_cost_per_released'] + Cohort_metrics['33 Collections_cost_per_released'] +Cohort_metrics['32 Reissunace_cost_per_released'] + Cohort_metrics['29 Partner_Bank_Bill_charging_cost_per_released'] + Cohort_metrics['28 Conductor_cost_per_released'] +Cohort_metrics['31 Sorocred_cost_per_released'])

    # Reflects current assumptions; Net Interest Income = Billed Income - Tax - Interest Expense (Assumed GS rate)
    Cohort_metrics['38 ADB_per_released'] = Cohort_metrics['adb_overall'] / Cohort_metrics['cust']
    Cohort_metrics['38 ADB_late_per_released'] = Cohort_metrics['late_adb'] / Cohort_metrics['cust']
    Cohort_metrics['38 ADB_revolve_per_released'] = Cohort_metrics['revolve_adb'] / Cohort_metrics['cust']
    Cohort_metrics['38 ADB_installments_per_released'] = Cohort_metrics['issuer_installments_adb'] / Cohort_metrics['cust']
    Cohort_metrics['38 ADB_other_per_released'] = Cohort_metrics['adb_other'] / Cohort_metrics['cust']
    Cohort_metrics['38a B_interest_per_released'] = Cohort_metrics['int_f_revenue'] / Cohort_metrics['cust']
    Cohort_metrics['38b B_lfee_per_released'] = Cohort_metrics['late_fee_f'] / Cohort_metrics['cust']
    Cohort_metrics['38c B_reward_fee_per_released'] = Cohort_metrics['rewards_fee_f'] / Cohort_metrics['cust']

    Cohort_metrics['39 Billed_interest_per_released'] = (Cohort_metrics['38a B_interest_per_released'] + Cohort_metrics['38b B_lfee_per_released'] + Cohort_metrics['38c B_reward_fee_per_released'])
    Cohort_metrics['40 Tax_on_interest_per_released'] = Cohort_metrics['39 Billed_interest_per_released'] * FINANCIAL_REVENUES_TAXES
    Cohort_metrics['41 Interest_Income_on_balances_per_released'] = Cohort_metrics['39 Billed_interest_per_released'] - Cohort_metrics['40 Tax_on_interest_per_released']
    Cohort_metrics['42 Post_Tax_Interest_Rate'] = Cohort_metrics['41 Interest_Income_on_balances_per_released'] / Cohort_metrics['38 ADB_per_released']
    Cohort_metrics['43 Interest_Expense_per_released'] = Cohort_metrics['38 ADB_per_released'] * COST_OF_DEBT_MONTHLY
    Cohort_metrics['44 Interest_Expense_per_ADB'] = Cohort_metrics['43 Interest_Expense_per_released'] / Cohort_metrics['38 ADB_per_released']
    Cohort_metrics['45 Net_Interest_Income_per_released'] = (Cohort_metrics['39 Billed_interest_per_released'] - Cohort_metrics['40 Tax_on_interest_per_released'] -Cohort_metrics['43 Interest_Expense_per_released'])

    #  dont use 46, 47, 48
    Cohort_metrics['46 Max_Lines_per_released'] = Cohort_metrics['max_credit_limit'] / Cohort_metrics['cust']

    Cohort_metrics['47 Chargeoffs_per_released'] = Cohort_metrics['CO_customers_this_mth_adj'] / Cohort_metrics['CO_metrics_denominator_customers']
    Cohort_metrics['48 Chargeoffs_Lines_per_released'] = Cohort_metrics['CO_Lines_adj'] / Cohort_metrics['CO_customers_this_mth_adj']

    # to be changed
    Cohort_metrics['49 Losses_per_released'] = Cohort_metrics['47 Chargeoffs_per_released'] * Cohort_metrics['48 Chargeoffs_Lines_per_released']

    Cohort_metrics['50 Recognized_Credit_Losses_per_released'] = Cohort_metrics['chargeoffamt_f'].fillna(0) / Cohort_metrics['cust']
    #     Cohort_metrics['losses_shifted_7'] = (Cohort_metrics['Current_Balance_Revolver']*0.0016 / Cohort_metrics['cust']) + (Cohort_metrics['Current_Balance_Transactor']*0.0016 / Cohort_metrics['cust'])
    #     Cohort_metrics['losses_shifted_6'] = Cohort_metrics['0-30 Balance']*0.1307 / Cohort_metrics['cust']
    #     Cohort_metrics['losses_shifted_5'] = Cohort_metrics['30-60 Balance']*0.4858 / Cohort_metrics['cust']
    #     Cohort_metrics['losses_shifted_4'] = Cohort_metrics['60-90 Balance']*0.7732 / Cohort_metrics['cust']
    #     Cohort_metrics['losses_shifted_3'] = Cohort_metrics['90-120 Balance']*0.8960 / Cohort_metrics['cust']
    #     Cohort_metrics['losses_shifted_2'] = Cohort_metrics['120-150 Balance']*.9589 / Cohort_metrics['cust']
    #     Cohort_metrics['losses_shifted_1'] = Cohort_metrics['150-180 Balance']*0.9798 / Cohort_metrics['cust']



    # Components of each bucket for provisions ----- Reflects hybrid provisions (Losses where data is available but provisioned rates where it is not)
    Cohort_metrics['losses_shifted_7'] = np.where(Cohort_metrics['chargeoffamt_f'].shift(-7).isnull() == True,Cohort_metrics['prov_for_current'] / Cohort_metrics['cust'],Cohort_metrics['chargeoffamt_f'].shift(-7) / Cohort_metrics['cust'])
    Cohort_metrics['losses_shifted_6'] = np.where(Cohort_metrics['chargeoffamt_f'].shift(-6).isnull() == True,Cohort_metrics['prov_for_0to30'] / Cohort_metrics['cust'],Cohort_metrics['chargeoffamt_f'].shift(-6) / Cohort_metrics['cust'])
    Cohort_metrics['losses_shifted_5'] = np.where(Cohort_metrics['chargeoffamt_f'].shift(-5).isnull() == True,Cohort_metrics['prov_for_30to60'] / Cohort_metrics['cust'],Cohort_metrics['chargeoffamt_f'].shift(-5) / Cohort_metrics['cust'])
    Cohort_metrics['losses_shifted_4'] = np.where(Cohort_metrics['chargeoffamt_f'].shift(-4).isnull() == True,Cohort_metrics['prov_for_60to90'] / Cohort_metrics['cust'],Cohort_metrics['chargeoffamt_f'].shift(-4) / Cohort_metrics['cust'])
    Cohort_metrics['losses_shifted_3'] = np.where(Cohort_metrics['chargeoffamt_f'].shift(-3).isnull() == True,Cohort_metrics['prov_for_90to120'] / Cohort_metrics['cust'],Cohort_metrics['chargeoffamt_f'].shift(-3) / Cohort_metrics['cust'])
    Cohort_metrics['losses_shifted_2'] = np.where(Cohort_metrics['chargeoffamt_f'].shift(-2).isnull() == True,Cohort_metrics['prov_for_120to150'] / Cohort_metrics['cust'],Cohort_metrics['chargeoffamt_f'].shift(-2) / Cohort_metrics['cust'])
    Cohort_metrics['losses_shifted_1'] = np.where(Cohort_metrics['chargeoffamt_f'].shift(-1).isnull() == True,Cohort_metrics['150-180 Balance'] * 0.9798 / Cohort_metrics['cust'],Cohort_metrics['chargeoffamt_f'].shift(-1) / Cohort_metrics['cust'])

    # Components of each bucket for provisions experimental ----- Reflects data based provisions (Losses where data is available but provisioned rates where it is not)
    Cohort_metrics['exp_losses_shifted_7'] = Cohort_metrics['prov_for_current'] / Cohort_metrics['cust']
    Cohort_metrics['exp_losses_shifted_6'] = Cohort_metrics['prov_for_0to30'] / Cohort_metrics['cust']
    Cohort_metrics['exp_losses_shifted_5'] = Cohort_metrics['prov_for_30to60'] / Cohort_metrics['cust']
    Cohort_metrics['exp_losses_shifted_4'] = Cohort_metrics['prov_for_60to90'] / Cohort_metrics['cust']
    Cohort_metrics['exp_losses_shifted_3'] = Cohort_metrics['prov_for_90to120'] / Cohort_metrics['cust']
    Cohort_metrics['exp_losses_shifted_2'] = Cohort_metrics['prov_for_120to150'] / Cohort_metrics['cust']
    Cohort_metrics['exp_losses_shifted_1'] = Cohort_metrics['150-180 Balance'] * 0.9798 / Cohort_metrics['cust']
    
    
    # Provisions section
    Cohort_metrics['51 Perfect_provisions_per_released'] = Cohort_metrics['losses_shifted_1'] + Cohort_metrics['losses_shifted_2'] + Cohort_metrics['losses_shifted_3'] + Cohort_metrics['losses_shifted_4'] + Cohort_metrics['losses_shifted_5'] + Cohort_metrics['losses_shifted_6'] + Cohort_metrics['losses_shifted_7']
    Cohort_metrics['51 absolute_provisions_per_released'] = Cohort_metrics['exp_losses_shifted_1'] + Cohort_metrics['exp_losses_shifted_2'] + Cohort_metrics['exp_losses_shifted_3'] + Cohort_metrics['exp_losses_shifted_4'] + Cohort_metrics['exp_losses_shifted_5'] + Cohort_metrics['exp_losses_shifted_6'] + Cohort_metrics['exp_losses_shifted_7']
    Cohort_metrics['52 Change_in_provisions_per_released'] = Cohort_metrics['51 Perfect_provisions_per_released'].diff().fillna(Cohort_metrics['51 Perfect_provisions_per_released'].values[0])


    Cohort_metrics['52a EOM outstanding_balance_per_released'] = Cohort_metrics['outstanding_balance'] / Cohort_metrics['cust']
    Cohort_metrics['52b Avg outstanding_balance_per_released'] = Cohort_metrics['avg_outstanding_bal'] / Cohort_metrics['cust']

    Cohort_metrics['53 Transaction_fraud_per_released'] = -Cohort_metrics['frd_f'] / Cohort_metrics['cust']
    Cohort_metrics['54 ID_fraud_per_released'] = -Cohort_metrics['id_frd_f'] / Cohort_metrics['cust']
    Cohort_metrics['54a Recovery_per_released'] = Cohort_metrics['credit_loss_recovery_f'] / Cohort_metrics['cust']


    # Corrent answer for recognized losse
    Cohort_metrics['55 Recognized_Losses_per_released'] = Cohort_metrics['54 ID_fraud_per_released'] + Cohort_metrics['53 Transaction_fraud_per_released'] - Cohort_metrics['54a Recovery_per_released'] + Cohort_metrics['50 Recognized_Credit_Losses_per_released']
    Cohort_metrics['56 Platinum_Risk_Adjusted_Margin'] = Cohort_metrics['45 Net_Interest_Income_per_released'] + Cohort_metrics['22 Platinum_Net_non_interest_income_per_released'] - Cohort_metrics['55 Recognized_Losses_per_released']
    Cohort_metrics['57 Gold_Risk_Adjusted_Margin'] = Cohort_metrics['45 Net_Interest_Income_per_released'] + Cohort_metrics['23 Gold_Net_non_interest_income_per_released'] - Cohort_metrics['55 Recognized_Losses_per_released']

    # Risk adjusted Margin = Net Interest Income + Net Non interest income - Recognized Losses
    Cohort_metrics['57a Risk_Adjusted_Margin'] = Cohort_metrics['45 Net_Interest_Income_per_released'] + Cohort_metrics['23a Net_non_interest_income_per_released'] - Cohort_metrics['55 Recognized_Losses_per_released']

    # Risk adjusted Margin incl provisions = Net Interest Income + Net Non interest income - Recognized Losses - Change in provisions
    Cohort_metrics['58 Platinum_Risk_Adjusted_Margin_incl_Provisions'] = Cohort_metrics['56 Platinum_Risk_Adjusted_Margin'] - Cohort_metrics['52 Change_in_provisions_per_released']
    Cohort_metrics['59 Gold_Risk_Adjusted_Margin_incl_Provisions'] = Cohort_metrics['57 Gold_Risk_Adjusted_Margin'] - Cohort_metrics['52 Change_in_provisions_per_released']
    Cohort_metrics['59a Risk_Adjusted_Margin_incl_Provisions'] = Cohort_metrics['57a Risk_Adjusted_Margin'] - Cohort_metrics['52 Change_in_provisions_per_released']

    # Contribution Margin = Risk adjusted margin - Non Interest Expense
    Cohort_metrics['60 Platinum_Contribution_Margin'] = Cohort_metrics['56 Platinum_Risk_Adjusted_Margin'] - Cohort_metrics['36 Platinum_Non_Interest_Expense_per_released']
    Cohort_metrics['61 Gold_Contribution_Margin'] = Cohort_metrics['57 Gold_Risk_Adjusted_Margin'] - Cohort_metrics['37 Gold_Non_Interest_Expense_per_released']
    Cohort_metrics['61a Contribution_Margin'] = Cohort_metrics['57a Risk_Adjusted_Margin'] - Cohort_metrics['37a Non_Interest_Expense_per_released']

    Cohort_metrics['62 Platinum_Contribution_Margin_incl_Provisions'] = Cohort_metrics['58 Platinum_Risk_Adjusted_Margin_incl_Provisions'] - Cohort_metrics['36 Platinum_Non_Interest_Expense_per_released']
    Cohort_metrics['63 Gold_Contribution_Margin_incl_Provisions'] = Cohort_metrics['59 Gold_Risk_Adjusted_Margin_incl_Provisions'] - Cohort_metrics['37 Gold_Non_Interest_Expense_per_released']
    Cohort_metrics['63a Contribution_Margin_incl_Provisions'] = Cohort_metrics['59a Risk_Adjusted_Margin_incl_Provisions'] - Cohort_metrics['37a Non_Interest_Expense_per_released']

    Cohort_metrics['64 Acquisition_costs'] = np.where(At_scale_costs == "Yes",np.where(Cohort_metrics.reset_index().mth == 0, ACQUISITION_COSTS_PER_RELEASED, 0) + ((Cohort_metrics['acq_cost'] -Cohort_metrics['Acq_Personnel_costs'] -Cohort_metrics['Mktg_costs']) /Cohort_metrics['cust']),(Cohort_metrics['acq_cost'] / Cohort_metrics['cust']))

    Cohort_metrics['65 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions'] = Cohort_metrics['62 Platinum_Contribution_Margin_incl_Provisions'] - Cohort_metrics['64 Acquisition_costs']
    Cohort_metrics['66 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions'] = Cohort_metrics['63 Gold_Contribution_Margin_incl_Provisions'] - Cohort_metrics['64 Acquisition_costs']
    Cohort_metrics['66a Contribution_Margin_incl_Provisions_w_Acquisitions'] = Cohort_metrics['63a Contribution_Margin_incl_Provisions'] - Cohort_metrics['64 Acquisition_costs']

    Cohort_metrics['67 Platinum_Contribution_Margin_w_Acquisitions'] = Cohort_metrics['60 Platinum_Contribution_Margin'] - Cohort_metrics['64 Acquisition_costs']
    Cohort_metrics['68 Gold_Contribution_Margin_w_Acquisitions'] = Cohort_metrics['61 Gold_Contribution_Margin'] - Cohort_metrics['64 Acquisition_costs']
    Cohort_metrics['68a Contribution_Margin_w_Acquisitions'] = Cohort_metrics['61a Contribution_Margin'] - Cohort_metrics['64 Acquisition_costs']

    Cohort_metrics['69 Platinum_Contribution_Margin_w_Acquisitions_cum'] = Cohort_metrics['67 Platinum_Contribution_Margin_w_Acquisitions'].cumsum()
    Cohort_metrics['70 Gold_Contribution_Margin_w_Acquisitions_cum'] = Cohort_metrics['68 Gold_Contribution_Margin_w_Acquisitions'].cumsum()
    Cohort_metrics['70a Contribution_Margin_w_Acquisitions_cum'] = Cohort_metrics['68a Contribution_Margin_w_Acquisitions'].cumsum()

    Cohort_metrics['71 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions_cum'] = Cohort_metrics['65 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions'].cumsum()
    Cohort_metrics['72 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions_cum'] = Cohort_metrics['66 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions'].cumsum()
    Cohort_metrics['72a Contribution_Margin_incl_Provisions_w_Acquisitions_cum'] = Cohort_metrics['66a Contribution_Margin_incl_Provisions_w_Acquisitions'].cumsum()

    # Operations below to output Lusa distibutions so that appropriate comparisons can be made with Forecasts after normalization
    print summ3[
        (summ3.cohort >= 201410) & (summ3.cohort.isin(ca)) & (summ3.exclude_censor == 0)].cohort.value_counts().size
    # print i
    Cohort_metrics['Type'] = 'Actuals'
    # Cohort_metrics['Vintages Included'] = 'All'

    Cust_Lusa_dist = summ3[(summ3.cohort >= 201410) & (summ3.cohort.isin(ca)) & (summ3.exclude_censor == 0)].groupby(['mth', 'lusa_band']).sum()[['cust', 'CO_metrics_denominator_customers', 'CO_customers_this_mth']].reset_index()
    Cust_Lusa_dist['cust_dist'] = Cust_Lusa_dist.groupby(['mth'])['cust'].apply(lambda x: x / float(x.sum()))
    Cust_Lusa_dist['CO_metrics_denominator_customers_dist'] = Cust_Lusa_dist.groupby(['mth'])['CO_metrics_denominator_customers'].apply(lambda x: x / float(x.sum()))
    Cust_Lusa_dist['CO_customers_this_mth_dist'] = Cust_Lusa_dist.groupby(['mth'])['CO_customers_this_mth'].apply(lambda x: x / float(x.sum()))

    Plat_dist = summ3[(summ3.cohort >= 201410) & (summ3.cohort.isin(ca)) & (summ3.exclude_censor == 0)].groupby(['mth']).sum()[['Platinum_custs', 'cust']].reset_index()
    Plat_dist['Plat_perc'] = Plat_dist['Platinum_custs'] / Plat_dist['cust']

    Cust_Lusa_dist = Cust_Lusa_dist.merge(Plat_dist[['mth', 'Plat_perc']], on='mth')

    return Cohort_metrics, Cust_Lusa_dist



In [19]:
def lapa(ledgers2, Summ_variables, base_line_lusa = 'rule 650+' ):
    # function does the same thing as cm but without adjusting for Lusa distribution. Instead it takes straight from the ledger on this
    Lusa_adjusted_performance = ledgers2[(ledgers2['Unnamed: 0'] == base_line_lusa) & (ledgers2['mth'] > -1)]
    Lusa_adjusted_performance['01 Purchase_gmv_per_released'] = Lusa_adjusted_performance['offb_opera_spending']
    Lusa_adjusted_performance['02 Purchase_igmv_per_released'] = Lusa_adjusted_performance['offb_opera_i_spending']

    #TODO WHAT IS THIS 0.95 CONSTANT? MOVE TO CONSTANTS.PY
    Lusa_adjusted_performance['03 Purchase_igmv_domestic_per_released'] = 0.95 * Lusa_adjusted_performance['offb_opera_i_spending']

    #TODO IS THERE A RESON TO HAVE A 0.0178 INSTEAD OF 0.0175?
    Lusa_adjusted_performance['04 Platinum_Gross_domestic_Interchange_revenue_per_released'] = 0.0178 * Lusa_adjusted_performance['03 Purchase_igmv_domestic_per_released']
    Lusa_adjusted_performance['05 Platinum_Taxes_domestic_Interchange_revenue_per_released'] =  OPERATING_REVENUES_TAXES * Lusa_adjusted_performance['04 Platinum_Gross_domestic_Interchange_revenue_per_released']
    Lusa_adjusted_performance['06 Platinum_Net_domestic_Interchange_revenue_per_released'] = Lusa_adjusted_performance['04 Platinum_Gross_domestic_Interchange_revenue_per_released'] - Lusa_adjusted_performance['05 Platinum_Taxes_domestic_Interchange_revenue_per_released']

    Lusa_adjusted_performance['07 Gold_Gross_domestic_Interchange_revenue_per_released'] = GOLD_DOMESTIC_INTERCHANGE_RATE * Lusa_adjusted_performance['03 Purchase_igmv_domestic_per_released']
    Lusa_adjusted_performance['08 Gold_Taxes_domestic_Interchange_revenue_per_released'] = OPERATING_REVENUES_TAXES * Lusa_adjusted_performance['07 Gold_Gross_domestic_Interchange_revenue_per_released']
    Lusa_adjusted_performance['09 Gold_Net_domestic_Interchange_revenue_per_released'] = Lusa_adjusted_performance['07 Gold_Gross_domestic_Interchange_revenue_per_released'] - Lusa_adjusted_performance['08 Gold_Taxes_domestic_Interchange_revenue_per_released']
    Lusa_adjusted_performance['07a Gross_domestic_Interchange_revenue_per_released'] = Lusa_adjusted_performance['07 Gold_Gross_domestic_Interchange_revenue_per_released']
    Lusa_adjusted_performance['08a Taxes_domestic_Interchange_revenue_per_released'] = Lusa_adjusted_performance['08 Gold_Taxes_domestic_Interchange_revenue_per_released']
    Lusa_adjusted_performance['09a Net_domestic_Interchange_revenue_per_released'] = Lusa_adjusted_performance['09 Gold_Net_domestic_Interchange_revenue_per_released']

    #TODO WHAT IS THIS 0.05? MOVE TO CONSTANTS.PY
    Lusa_adjusted_performance['10 Purchase_igmv_international_per_released'] = 0.05 * Lusa_adjusted_performance['offb_opera_i_spending']
    Lusa_adjusted_performance['11 Purchase_igmv_international_interchange_per_released'] = FOREIGN_INTERCHANGE_RATE * Lusa_adjusted_performance['10 Purchase_igmv_international_per_released']
    Lusa_adjusted_performance['12 Purchase_igmv_international_Taxes_interchange_per_released'] = OPERATING_REVENUES_TAXES * Lusa_adjusted_performance['11 Purchase_igmv_international_interchange_per_released']
    Lusa_adjusted_performance['13 Net_international_interchange_per_released'] = Lusa_adjusted_performance['11 Purchase_igmv_international_interchange_per_released'] - Lusa_adjusted_performance['12 Purchase_igmv_international_Taxes_interchange_per_released']

    Lusa_adjusted_performance['14 fx_spread_per_released'] = -Lusa_adjusted_performance['fx_spread']
    Lusa_adjusted_performance['15 fx_spread_taxes_per_released'] = Lusa_adjusted_performance['fx_spread_taxes']
    Lusa_adjusted_performance['16 net_fx_spread_per_released'] = Lusa_adjusted_performance['14 fx_spread_per_released'] - Lusa_adjusted_performance['15 fx_spread_taxes_per_released']
    Lusa_adjusted_performance['17 net_fx_rate'] = Lusa_adjusted_performance['16 net_fx_spread_per_released'] / Lusa_adjusted_performance['02 Purchase_igmv_per_released']

    Lusa_adjusted_performance['18 MC_spend_linked_domestic_per_released'] = MC_SPEND_LINKED_DOMESTIC_RATE * Lusa_adjusted_performance['03 Purchase_igmv_domestic_per_released']
    Lusa_adjusted_performance['19 MC_spend_linked_international_per_released'] = MC_SPEND_LINKED_FOREIGN_RATE * Lusa_adjusted_performance['10 Purchase_igmv_international_per_released']
    Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released'] = Lusa_adjusted_performance['18 MC_spend_linked_domestic_per_released'] + Lusa_adjusted_performance['19 MC_spend_linked_international_per_released']
    Lusa_adjusted_performance['21 MC_spend_linked_costs_rate'] = Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released'] / Lusa_adjusted_performance['02 Purchase_igmv_per_released']

    Lusa_adjusted_performance['22 Platinum_Net_non_interest_income_per_released'] = Lusa_adjusted_performance['06 Platinum_Net_domestic_Interchange_revenue_per_released'] + Lusa_adjusted_performance['13 Net_international_interchange_per_released'] + Lusa_adjusted_performance['16 net_fx_spread_per_released'] - Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released']
    Lusa_adjusted_performance['23 Gold_Net_non_interest_income_per_released'] = Lusa_adjusted_performance['09 Gold_Net_domestic_Interchange_revenue_per_released'] + Lusa_adjusted_performance['13 Net_international_interchange_per_released'] + Lusa_adjusted_performance['16 net_fx_spread_per_released'] - Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released']
    Lusa_adjusted_performance['23a Net_non_interest_income_per_released'] = Lusa_adjusted_performance['09a Net_domestic_Interchange_revenue_per_released'] + Lusa_adjusted_performance['13 Net_international_interchange_per_released'] + Lusa_adjusted_performance['16 net_fx_spread_per_released'] - Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released']
    Lusa_adjusted_performance['24 Platinum_Interchange_rate'] = (Lusa_adjusted_performance['06 Platinum_Net_domestic_Interchange_revenue_per_released'] + Lusa_adjusted_performance['13 Net_international_interchange_per_released']) / Lusa_adjusted_performance['02 Purchase_igmv_per_released']
    Lusa_adjusted_performance['25 Gold_Interchange_rate'] = (Lusa_adjusted_performance['09 Gold_Net_domestic_Interchange_revenue_per_released'] + Lusa_adjusted_performance['13 Net_international_interchange_per_released']) / Lusa_adjusted_performance['02 Purchase_igmv_per_released']
    Lusa_adjusted_performance['25a Interchange_rate'] = Lusa_adjusted_performance['25 Gold_Interchange_rate']

    #TODO WHY IS THIS 3.5 INSTEAD OF 4.2? MOVE TO CONSTANTS.PY
    Lusa_adjusted_performance['26 Platinum_MC_account_linked_cost_per_released'] = 3.5
    Lusa_adjusted_performance['27 Gold_MC_account_linked_cost_per_released'] = MC_GOLD_CARD_LINKED_COST
    Lusa_adjusted_performance['27a MC_account_linked_cost_per_released'] = Lusa_adjusted_performance['27 Gold_MC_account_linked_cost_per_released']

    #SAME THING, DEFINE GLOBAL VARIABLE ON CONSTANTS.PY - MAKE CLEAR DIFFERENCES FROM OTHER FILE
    Lusa_adjusted_performance['28 Conductor_cost_per_released'] = CONDUCTOR_COST_PER_RELEASED
    Lusa_adjusted_performance['29 Partner_Bank_Bill_charging_cost_per_released'] = 1.1  # 1.24 per payment

    Lusa_adjusted_performance['32 Reissunace_cost_per_released'] = 0.1
    Lusa_adjusted_performance['33 Collections_cost_per_released'] = 0.5
    Lusa_adjusted_performance['34 Cust_Mgmt_cost_per_released'] = 1
    Lusa_adjusted_performance['35 Tech_Comm_cost_per_released'] = 0.1

    Lusa_adjusted_performance['38 ADB_per_released'] = Lusa_adjusted_performance['main_asset_balance']
    Lusa_adjusted_performance['38a B_interest_per_released'] = -Lusa_adjusted_performance['interest_revenues']
    Lusa_adjusted_performance['38b B_lfee_per_released'] = -Lusa_adjusted_performance['lfee_revenues']


    Lusa_adjusted_performance['39 Billed_interest_per_released'] = -Lusa_adjusted_performance['interest_revenues'] - Lusa_adjusted_performance['lfee_revenues']
    Lusa_adjusted_performance['40 Tax_on_interest_per_released'] = Lusa_adjusted_performance['interest_taxes'] + Lusa_adjusted_performance['lfee_taxes']
    Lusa_adjusted_performance['41 Interest_Income_on_balances_per_released'] = Lusa_adjusted_performance['39 Billed_interest_per_released'] - Lusa_adjusted_performance['40 Tax_on_interest_per_released']
    Lusa_adjusted_performance['42 Post_Tax_Interest_Rate'] = Lusa_adjusted_performance['41 Interest_Income_on_balances_per_released'] / Lusa_adjusted_performance['38 ADB_per_released']
    Lusa_adjusted_performance['43 Interest_Expense_per_released'] = Lusa_adjusted_performance['interest_expense']
    Lusa_adjusted_performance['44 Interest_Expense_per_ADB'] = Lusa_adjusted_performance['43 Interest_Expense_per_released'] / Lusa_adjusted_performance['38 ADB_per_released']
    Lusa_adjusted_performance['45 Net_Interest_Income_per_released'] = (Lusa_adjusted_performance['41 Interest_Income_on_balances_per_released'] - Lusa_adjusted_performance['43 Interest_Expense_per_released'])

    Lusa_adjusted_performance['30 Bracce_cost_per_released'] = 0.006 * 1.46 * Lusa_adjusted_performance['38 ADB_per_released']
    Lusa_adjusted_performance['31 Sorocred_cost_per_released'] = 0.003 * 1.46 * Lusa_adjusted_performance['38 ADB_per_released']

    Lusa_adjusted_performance['36 Platinum_Non_Interest_Expense_per_released'] = (Lusa_adjusted_performance['31 Sorocred_cost_per_released'] + Lusa_adjusted_performance['26 Platinum_MC_account_linked_cost_per_released'] + Lusa_adjusted_performance['35 Tech_Comm_cost_per_released'] + Lusa_adjusted_performance['34 Cust_Mgmt_cost_per_released'] +Lusa_adjusted_performance['33 Collections_cost_per_released'] + Lusa_adjusted_performance['32 Reissunace_cost_per_released'] + Lusa_adjusted_performance['29 Partner_Bank_Bill_charging_cost_per_released'] + Lusa_adjusted_performance['28 Conductor_cost_per_released'])
    Lusa_adjusted_performance['37 Gold_Non_Interest_Expense_per_released'] = (Lusa_adjusted_performance['31 Sorocred_cost_per_released'] + Lusa_adjusted_performance['27 Gold_MC_account_linked_cost_per_released'] + Lusa_adjusted_performance['35 Tech_Comm_cost_per_released'] +Lusa_adjusted_performance['34 Cust_Mgmt_cost_per_released'] + Lusa_adjusted_performance['33 Collections_cost_per_released'] + Lusa_adjusted_performance['32 Reissunace_cost_per_released'] +Lusa_adjusted_performance['29 Partner_Bank_Bill_charging_cost_per_released'] + Lusa_adjusted_performance['28 Conductor_cost_per_released'])
    Lusa_adjusted_performance['37a Non_Interest_Expense_per_released'] = Lusa_adjusted_performance['37 Gold_Non_Interest_Expense_per_released']

    Lusa_adjusted_performance['46 Max_Lines_per_released'] = -Lusa_adjusted_performance['crli_liabi_released']
    Lusa_adjusted_performance['47 Chargeoffs_per_released'] = Lusa_adjusted_performance['CO']
    Lusa_adjusted_performance['48 Chargeoffs_Lines_per_released'] = (Lusa_adjusted_performance['losses_shifted'] / Lusa_adjusted_performance['CO_shifted'])

    Lusa_adjusted_performance['49 Losses_per_released'] = Lusa_adjusted_performance['losses_shifted']
    Lusa_adjusted_performance['50 Recognized_Credit_Losses_per_released'] = Lusa_adjusted_performance['losses']

    Lusa_adjusted_performance['51 Perfect_provisions_per_released'] = Lusa_adjusted_performance['Perfect_provision']
    Lusa_adjusted_performance['52 Change_in_provisions_per_released'] = Lusa_adjusted_performance['Change_in_Provision']

    Lusa_adjusted_performance['53 Transaction_fraud_per_released'] = Lusa_adjusted_performance['txn_frd']
    Lusa_adjusted_performance['54 ID_fraud_per_released'] = 0
    Lusa_adjusted_performance['54a Recovery_per_released'] = -Lusa_adjusted_performance['credit_recoveries']

    Lusa_adjusted_performance['55 Recognized_Losses_per_released'] = Lusa_adjusted_performance['54 ID_fraud_per_released'] + Lusa_adjusted_performance['53 Transaction_fraud_per_released'] + Lusa_adjusted_performance['50 Recognized_Credit_Losses_per_released'] - Lusa_adjusted_performance['54a Recovery_per_released']
    Lusa_adjusted_performance['56 Platinum_Risk_Adjusted_Margin'] = Lusa_adjusted_performance['45 Net_Interest_Income_per_released'] + Lusa_adjusted_performance['22 Platinum_Net_non_interest_income_per_released'] - Lusa_adjusted_performance['55 Recognized_Losses_per_released']
    Lusa_adjusted_performance['57 Gold_Risk_Adjusted_Margin'] = Lusa_adjusted_performance['45 Net_Interest_Income_per_released'] + Lusa_adjusted_performance['23 Gold_Net_non_interest_income_per_released'] - Lusa_adjusted_performance['55 Recognized_Losses_per_released']
    Lusa_adjusted_performance['57a Risk_Adjusted_Margin'] = Lusa_adjusted_performance['57 Gold_Risk_Adjusted_Margin']

    Lusa_adjusted_performance['58 Platinum_Risk_Adjusted_Margin_incl_Provisions'] = Lusa_adjusted_performance['56 Platinum_Risk_Adjusted_Margin'] - Lusa_adjusted_performance['52 Change_in_provisions_per_released']
    Lusa_adjusted_performance['59 Gold_Risk_Adjusted_Margin_incl_Provisions'] = Lusa_adjusted_performance['57 Gold_Risk_Adjusted_Margin'] - Lusa_adjusted_performance['52 Change_in_provisions_per_released']
    Lusa_adjusted_performance['59a Risk_Adjusted_Margin_incl_Provisions'] = Lusa_adjusted_performance['59 Gold_Risk_Adjusted_Margin_incl_Provisions']

    Lusa_adjusted_performance['60 Platinum_Contribution_Margin'] = Lusa_adjusted_performance['56 Platinum_Risk_Adjusted_Margin'] - Lusa_adjusted_performance['36 Platinum_Non_Interest_Expense_per_released']
    Lusa_adjusted_performance['61 Gold_Contribution_Margin'] = Lusa_adjusted_performance['57 Gold_Risk_Adjusted_Margin'] - Lusa_adjusted_performance['37 Gold_Non_Interest_Expense_per_released']
    Lusa_adjusted_performance['61a Contribution_Margin'] = Lusa_adjusted_performance['61 Gold_Contribution_Margin']

    Lusa_adjusted_performance['62 Platinum_Contribution_Margin_incl_Provisions'] = Lusa_adjusted_performance['58 Platinum_Risk_Adjusted_Margin_incl_Provisions'] - Lusa_adjusted_performance['36 Platinum_Non_Interest_Expense_per_released']
    Lusa_adjusted_performance['63 Gold_Contribution_Margin_incl_Provisions'] = Lusa_adjusted_performance['59 Gold_Risk_Adjusted_Margin_incl_Provisions'] - Lusa_adjusted_performance['37 Gold_Non_Interest_Expense_per_released']
    Lusa_adjusted_performance['63a Contribution_Margin_incl_Provisions'] = Lusa_adjusted_performance['63 Gold_Contribution_Margin_incl_Provisions']

    Lusa_adjusted_performance['64 Acquisition_costs'] = np.where(Lusa_adjusted_performance['mth'] == 0, 28.9, 0)

    Lusa_adjusted_performance['65 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions'] = Lusa_adjusted_performance['62 Platinum_Contribution_Margin_incl_Provisions'] - Lusa_adjusted_performance['64 Acquisition_costs']
    Lusa_adjusted_performance['66 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions'] = Lusa_adjusted_performance['63 Gold_Contribution_Margin_incl_Provisions'] - Lusa_adjusted_performance['64 Acquisition_costs']
    Lusa_adjusted_performance['66a Contribution_Margin_incl_Provisions_w_Acquisitions'] = Lusa_adjusted_performance['66 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions']

    Lusa_adjusted_performance['67 Platinum_Contribution_Margin_w_Acquisitions'] = Lusa_adjusted_performance['60 Platinum_Contribution_Margin'] - Lusa_adjusted_performance['64 Acquisition_costs']
    Lusa_adjusted_performance['68 Gold_Contribution_Margin_w_Acquisitions'] = Lusa_adjusted_performance['61 Gold_Contribution_Margin'] - Lusa_adjusted_performance['64 Acquisition_costs']
    Lusa_adjusted_performance['68a Contribution_Margin_w_Acquisitions'] = Lusa_adjusted_performance['68 Gold_Contribution_Margin_w_Acquisitions']

    Lusa_adjusted_performance['69 Platinum_Contribution_Margin_w_Acquisitions_cum'] = Lusa_adjusted_performance['67 Platinum_Contribution_Margin_w_Acquisitions'].cumsum()
    Lusa_adjusted_performance['70 Gold_Contribution_Margin_w_Acquisitions_cum'] = Lusa_adjusted_performance['68 Gold_Contribution_Margin_w_Acquisitions'].cumsum()
    Lusa_adjusted_performance['70a Contribution_Margin_w_Acquisitions_cum'] = Lusa_adjusted_performance['68a Contribution_Margin_w_Acquisitions'].cumsum()

    Lusa_adjusted_performance['71 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions_cum'] = \
    Lusa_adjusted_performance['65 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions'].cumsum()
    Lusa_adjusted_performance['72 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions_cum'] = \
    Lusa_adjusted_performance['66 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions'].cumsum()
    Lusa_adjusted_performance['72a Contribution_Margin_incl_Provisions_w_Acquisitions_cum'] = Lusa_adjusted_performance['66a Contribution_Margin_incl_Provisions_w_Acquisitions'].cumsum()

    Lusa_adjusted_performance_summa = Lusa_adjusted_performance.groupby(['mth']).sum()[Summ_variables]
    Lusa_adjusted_performance_summa['Type'] = 'Forecast - Lusa 650+'
    # Lusa_adjusted_performance_summ['Vintages Included'] = 'All'
    # return Lusa_adjusted_performance
    return Lusa_adjusted_performance_summa


In [20]:
def lap(ledgers2, Cust_Lusa_dist, Summ_variables):
    # takes the placed ledger and does the same calculations normalizing for lusa based on actual observed lusa dist for cohorts

    #TODO SAME THING, MOVE CONSTANTS TO SEPARATE FILE
    Lusa_adjusted_performance = pd.merge(ledgers2, Cust_Lusa_dist, on=['mth', 'lusa_band'])
    Lusa_adjusted_performance['01 Purchase_gmv_per_released'] = Lusa_adjusted_performance['offb_opera_spending'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['02 Purchase_igmv_per_released'] = Lusa_adjusted_performance['offb_opera_i_spending'] * Lusa_adjusted_performance['cust_dist']

    Lusa_adjusted_performance['03 Purchase_igmv_domestic_per_released'] = 0.95 * Lusa_adjusted_performance['offb_opera_i_spending'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['04 Platinum_Gross_domestic_Interchange_revenue_per_released'] = 0.0178 * Lusa_adjusted_performance['03 Purchase_igmv_domestic_per_released']
    Lusa_adjusted_performance['05 Platinum_Taxes_domestic_Interchange_revenue_per_released'] = 0.1125 * Lusa_adjusted_performance['04 Platinum_Gross_domestic_Interchange_revenue_per_released']
    Lusa_adjusted_performance['06 Platinum_Net_domestic_Interchange_revenue_per_released'] = Lusa_adjusted_performance['04 Platinum_Gross_domestic_Interchange_revenue_per_released'] - Lusa_adjusted_performance['05 Platinum_Taxes_domestic_Interchange_revenue_per_released']

    Lusa_adjusted_performance['07 Gold_Gross_domestic_Interchange_revenue_per_released'] = 0.015 * Lusa_adjusted_performance['03 Purchase_igmv_domestic_per_released']
    Lusa_adjusted_performance['08 Gold_Taxes_domestic_Interchange_revenue_per_released'] = 0.1125 * Lusa_adjusted_performance['07 Gold_Gross_domestic_Interchange_revenue_per_released']
    Lusa_adjusted_performance['09 Gold_Net_domestic_Interchange_revenue_per_released'] = Lusa_adjusted_performance['07 Gold_Gross_domestic_Interchange_revenue_per_released'] - Lusa_adjusted_performance['08 Gold_Taxes_domestic_Interchange_revenue_per_released']
    Lusa_adjusted_performance['07a Gross_domestic_Interchange_revenue_per_released'] = (Lusa_adjusted_performance['Plat_perc'] *Lusa_adjusted_performance['04 Platinum_Gross_domestic_Interchange_revenue_per_released']) + ((1 - Lusa_adjusted_performance['Plat_perc']) *Lusa_adjusted_performance['07 Gold_Gross_domestic_Interchange_revenue_per_released'])
    Lusa_adjusted_performance['08a Taxes_domestic_Interchange_revenue_per_released'] = 0.1125 * Lusa_adjusted_performance['07a Gross_domestic_Interchange_revenue_per_released']
    Lusa_adjusted_performance['09a Net_domestic_Interchange_revenue_per_released'] = Lusa_adjusted_performance['07a Gross_domestic_Interchange_revenue_per_released'] - Lusa_adjusted_performance['08a Taxes_domestic_Interchange_revenue_per_released']

    Lusa_adjusted_performance['10 Purchase_igmv_international_per_released'] = 0.05 * Lusa_adjusted_performance['offb_opera_i_spending'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['11 Purchase_igmv_international_interchange_per_released'] = 0.0175 * Lusa_adjusted_performance['10 Purchase_igmv_international_per_released']
    Lusa_adjusted_performance['12 Purchase_igmv_international_Taxes_interchange_per_released'] = 0.1125 * Lusa_adjusted_performance['11 Purchase_igmv_international_interchange_per_released']
    Lusa_adjusted_performance['13 Net_international_interchange_per_released'] = Lusa_adjusted_performance['11 Purchase_igmv_international_interchange_per_released'] - Lusa_adjusted_performance['12 Purchase_igmv_international_Taxes_interchange_per_released']

    Lusa_adjusted_performance['14 fx_spread_per_released'] = -Lusa_adjusted_performance['fx_spread'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['15 fx_spread_taxes_per_released'] = Lusa_adjusted_performance['fx_spread_taxes'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['16 net_fx_spread_per_released'] = Lusa_adjusted_performance['14 fx_spread_per_released'] - Lusa_adjusted_performance['15 fx_spread_taxes_per_released']
    Lusa_adjusted_performance['17 net_fx_rate'] = Lusa_adjusted_performance['16 net_fx_spread_per_released'] / Lusa_adjusted_performance['02 Purchase_igmv_per_released']

    Lusa_adjusted_performance['18 MC_spend_linked_domestic_per_released'] = 0.75 * 0.0024 * Lusa_adjusted_performance['03 Purchase_igmv_domestic_per_released']
    Lusa_adjusted_performance['19 MC_spend_linked_international_per_released'] = 0.75 * 0.0223 * Lusa_adjusted_performance['10 Purchase_igmv_international_per_released']
    Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released'] = Lusa_adjusted_performance['18 MC_spend_linked_domestic_per_released'] + Lusa_adjusted_performance['19 MC_spend_linked_international_per_released']
    Lusa_adjusted_performance['21 MC_spend_linked_costs_rate'] = Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released'] / Lusa_adjusted_performance['02 Purchase_igmv_per_released']

    Lusa_adjusted_performance['22 Platinum_Net_non_interest_income_per_released'] = Lusa_adjusted_performance['06 Platinum_Net_domestic_Interchange_revenue_per_released'] + Lusa_adjusted_performance['13 Net_international_interchange_per_released'] + Lusa_adjusted_performance['16 net_fx_spread_per_released'] - Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released']
    Lusa_adjusted_performance['23 Gold_Net_non_interest_income_per_released'] = Lusa_adjusted_performance['09 Gold_Net_domestic_Interchange_revenue_per_released'] + Lusa_adjusted_performance['13 Net_international_interchange_per_released'] + Lusa_adjusted_performance['16 net_fx_spread_per_released'] - Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released']
    Lusa_adjusted_performance['23a Net_non_interest_income_per_released'] = Lusa_adjusted_performance['09a Net_domestic_Interchange_revenue_per_released'] + Lusa_adjusted_performance['13 Net_international_interchange_per_released'] + Lusa_adjusted_performance['16 net_fx_spread_per_released'] - Lusa_adjusted_performance['20 MC_spend_linked_costs_per_released']

    Lusa_adjusted_performance['24 Platinum_Interchange_rate'] = (Lusa_adjusted_performance['06 Platinum_Net_domestic_Interchange_revenue_per_released'] +Lusa_adjusted_performance['13 Net_international_interchange_per_released']) / Lusa_adjusted_performance['02 Purchase_igmv_per_released']
    Lusa_adjusted_performance['25 Gold_Interchange_rate'] = (Lusa_adjusted_performance['09 Gold_Net_domestic_Interchange_revenue_per_released'] +Lusa_adjusted_performance['13 Net_international_interchange_per_released']) / Lusa_adjusted_performance['02 Purchase_igmv_per_released']
    Lusa_adjusted_performance['25a Interchange_rate'] = (Lusa_adjusted_performance['Plat_perc'] *Lusa_adjusted_performance['24 Platinum_Interchange_rate']) + ((1 - Lusa_adjusted_performance['Plat_perc']) *Lusa_adjusted_performance['25 Gold_Interchange_rate'])

    Lusa_adjusted_performance['26 Platinum_MC_account_linked_cost_per_released'] = 3.5 * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['27 Gold_MC_account_linked_cost_per_released'] = 0.6 * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['27a MC_account_linked_cost_per_released'] = (Lusa_adjusted_performance['Plat_perc'] *Lusa_adjusted_performance['26 Platinum_MC_account_linked_cost_per_released']) + ((1 - Lusa_adjusted_performance['Plat_perc']) *Lusa_adjusted_performance['27 Gold_MC_account_linked_cost_per_released'])

    Lusa_adjusted_performance['28 Conductor_cost_per_released'] = 1.1 * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['29 Partner_Bank_Bill_charging_cost_per_released'] = 1.1 * Lusa_adjusted_performance['cust_dist']  # 1.24 per payment

    Lusa_adjusted_performance['32 Reissunace_cost_per_released'] = 0.1 * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['33 Collections_cost_per_released'] = 0.5 * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['34 Cust_Mgmt_cost_per_released'] = 1 * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['35 Tech_Comm_cost_per_released'] = 0.1 * Lusa_adjusted_performance['cust_dist']

    Lusa_adjusted_performance['38 ADB_per_released'] = Lusa_adjusted_performance['main_asset_balance'] * Lusa_adjusted_performance['cust_dist']
    
    Lusa_adjusted_performance['38a B_interest_per_released'] = -Lusa_adjusted_performance['interest_revenues'] * Lusa_adjusted_performance['cust_dist']
    #Needs changing
    Lusa_adjusted_performance['38b B_lfee_per_released'] = -Lusa_adjusted_performance['lfee_revenues'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['38c B_reward_fee_per_released'] = 0
    
    Lusa_adjusted_performance['39 Billed_interest_per_released'] = (Lusa_adjusted_performance['38a B_interest_per_released']  + Lusa_adjusted_performance['38b B_lfee_per_released'] + Lusa_adjusted_performance['38c B_reward_fee_per_released'])
    Lusa_adjusted_performance['40 Tax_on_interest_per_released'] = Lusa_adjusted_performance['interest_taxes'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['41 Interest_Income_on_balances_per_released'] = Lusa_adjusted_performance['39 Billed_interest_per_released'] - Lusa_adjusted_performance['40 Tax_on_interest_per_released']
    Lusa_adjusted_performance['42 Post_Tax_Interest_Rate'] = Lusa_adjusted_performance['41 Interest_Income_on_balances_per_released'] / Lusa_adjusted_performance['38 ADB_per_released']
    Lusa_adjusted_performance['43 Interest_Expense_per_released'] = Lusa_adjusted_performance['interest_expense'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['44 Interest_Expense_per_ADB'] = Lusa_adjusted_performance['43 Interest_Expense_per_released'] / Lusa_adjusted_performance['38 ADB_per_released']
    Lusa_adjusted_performance['45 Net_Interest_Income_per_released'] = (Lusa_adjusted_performance['41 Interest_Income_on_balances_per_released'] - Lusa_adjusted_performance['43 Interest_Expense_per_released'])

    Lusa_adjusted_performance['30 Bracce_cost_per_released'] = 0.006 * 1.46 * Lusa_adjusted_performance['38 ADB_per_released']
    Lusa_adjusted_performance['31 Sorocred_cost_per_released'] = 0.003 * 1.46 * Lusa_adjusted_performance['38 ADB_per_released']

    Lusa_adjusted_performance['36 Platinum_Non_Interest_Expense_per_released'] = (Lusa_adjusted_performance['31 Sorocred_cost_per_released'] + Lusa_adjusted_performance['26 Platinum_MC_account_linked_cost_per_released'] + Lusa_adjusted_performance['35 Tech_Comm_cost_per_released'] + Lusa_adjusted_performance['34 Cust_Mgmt_cost_per_released'] + Lusa_adjusted_performance['33 Collections_cost_per_released'] + Lusa_adjusted_performance['32 Reissunace_cost_per_released'] + Lusa_adjusted_performance['29 Partner_Bank_Bill_charging_cost_per_released'] + Lusa_adjusted_performance['28 Conductor_cost_per_released'])
    Lusa_adjusted_performance['37 Gold_Non_Interest_Expense_per_released'] = (Lusa_adjusted_performance['31 Sorocred_cost_per_released'] + Lusa_adjusted_performance['27 Gold_MC_account_linked_cost_per_released'] + Lusa_adjusted_performance['35 Tech_Comm_cost_per_released'] +Lusa_adjusted_performance['34 Cust_Mgmt_cost_per_released'] + Lusa_adjusted_performance['33 Collections_cost_per_released'] + Lusa_adjusted_performance['32 Reissunace_cost_per_released'] +Lusa_adjusted_performance['29 Partner_Bank_Bill_charging_cost_per_released'] + Lusa_adjusted_performance['28 Conductor_cost_per_released'])
    Lusa_adjusted_performance['37a Non_Interest_Expense_per_released'] = (Lusa_adjusted_performance['31 Sorocred_cost_per_released'] + Lusa_adjusted_performance['27a MC_account_linked_cost_per_released'] + Lusa_adjusted_performance['35 Tech_Comm_cost_per_released'] +Lusa_adjusted_performance['34 Cust_Mgmt_cost_per_released'] + Lusa_adjusted_performance['33 Collections_cost_per_released'] + Lusa_adjusted_performance['32 Reissunace_cost_per_released'] +Lusa_adjusted_performance['29 Partner_Bank_Bill_charging_cost_per_released'] + Lusa_adjusted_performance['28 Conductor_cost_per_released'])
    

    Lusa_adjusted_performance['46 Max_Lines_per_released'] = -Lusa_adjusted_performance['crli_liabi_released'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['47 Chargeoffs_per_released'] = Lusa_adjusted_performance['CO'] * Lusa_adjusted_performance['CO_metrics_denominator_customers_dist']
    Lusa_adjusted_performance['48 Chargeoffs_Lines_per_released'] = (Lusa_adjusted_performance['losses_shifted'] /Lusa_adjusted_performance['CO_shifted']) * Lusa_adjusted_performance['CO_customers_this_mth_dist']

    Lusa_adjusted_performance['49 Losses_per_released'] = Lusa_adjusted_performance['losses_shifted'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['50 Recognized_Credit_Losses_per_released'] = Lusa_adjusted_performance['losses'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['51 Perfect_provisions_per_released'] = Lusa_adjusted_performance['Perfect_provision'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['52 Change_in_provisions_per_released'] = Lusa_adjusted_performance['Change_in_Provision'] * Lusa_adjusted_performance['cust_dist']

    Lusa_adjusted_performance['53 Transaction_fraud_per_released'] = Lusa_adjusted_performance['txn_frd'] * Lusa_adjusted_performance['cust_dist']
    Lusa_adjusted_performance['54 ID_fraud_per_released'] = 0
    Lusa_adjusted_performance['54a Recovery_per_released'] = -Lusa_adjusted_performance['credit_recoveries']* Lusa_adjusted_performance['cust_dist']
    
    Lusa_adjusted_performance['55 Recognized_Losses_per_released'] = Lusa_adjusted_performance['54 ID_fraud_per_released'] + Lusa_adjusted_performance['53 Transaction_fraud_per_released'] + Lusa_adjusted_performance['50 Recognized_Credit_Losses_per_released'] - Lusa_adjusted_performance['54a Recovery_per_released']
    Lusa_adjusted_performance['56 Platinum_Risk_Adjusted_Margin'] = Lusa_adjusted_performance['45 Net_Interest_Income_per_released'] + Lusa_adjusted_performance['22 Platinum_Net_non_interest_income_per_released'] - Lusa_adjusted_performance['55 Recognized_Losses_per_released']
    Lusa_adjusted_performance['57 Gold_Risk_Adjusted_Margin'] = Lusa_adjusted_performance['45 Net_Interest_Income_per_released'] + Lusa_adjusted_performance['23 Gold_Net_non_interest_income_per_released'] - Lusa_adjusted_performance['55 Recognized_Losses_per_released']
    Lusa_adjusted_performance['57a Risk_Adjusted_Margin'] = Lusa_adjusted_performance['45 Net_Interest_Income_per_released'] + Lusa_adjusted_performance['23a Net_non_interest_income_per_released'] - Lusa_adjusted_performance['55 Recognized_Losses_per_released']

    Lusa_adjusted_performance['58 Platinum_Risk_Adjusted_Margin_incl_Provisions'] = Lusa_adjusted_performance['56 Platinum_Risk_Adjusted_Margin'] - Lusa_adjusted_performance['52 Change_in_provisions_per_released']
    Lusa_adjusted_performance['59 Gold_Risk_Adjusted_Margin_incl_Provisions'] = Lusa_adjusted_performance['57 Gold_Risk_Adjusted_Margin'] - Lusa_adjusted_performance['52 Change_in_provisions_per_released']
    Lusa_adjusted_performance['59a Risk_Adjusted_Margin_incl_Provisions'] = Lusa_adjusted_performance['57a Risk_Adjusted_Margin'] - Lusa_adjusted_performance['52 Change_in_provisions_per_released']

    Lusa_adjusted_performance['60 Platinum_Contribution_Margin'] = Lusa_adjusted_performance['56 Platinum_Risk_Adjusted_Margin'] - Lusa_adjusted_performance['36 Platinum_Non_Interest_Expense_per_released']
    Lusa_adjusted_performance['61 Gold_Contribution_Margin'] = Lusa_adjusted_performance['57 Gold_Risk_Adjusted_Margin'] - Lusa_adjusted_performance['37 Gold_Non_Interest_Expense_per_released']
    Lusa_adjusted_performance['61a Contribution_Margin'] = Lusa_adjusted_performance['57a Risk_Adjusted_Margin'] - Lusa_adjusted_performance['37a Non_Interest_Expense_per_released']

    Lusa_adjusted_performance['62 Platinum_Contribution_Margin_incl_Provisions'] = Lusa_adjusted_performance['58 Platinum_Risk_Adjusted_Margin_incl_Provisions'] - Lusa_adjusted_performance['36 Platinum_Non_Interest_Expense_per_released']
    Lusa_adjusted_performance['63 Gold_Contribution_Margin_incl_Provisions'] = Lusa_adjusted_performance['59 Gold_Risk_Adjusted_Margin_incl_Provisions'] - Lusa_adjusted_performance['37 Gold_Non_Interest_Expense_per_released']
    Lusa_adjusted_performance['63a Contribution_Margin_incl_Provisions'] = Lusa_adjusted_performance['59a Risk_Adjusted_Margin_incl_Provisions'] - Lusa_adjusted_performance['37a Non_Interest_Expense_per_released']

    Lusa_adjusted_performance['64 Acquisition_costs'] = np.where(Lusa_adjusted_performance['mth'] == 0,28.9 * Lusa_adjusted_performance['cust_dist'], 0)

    Lusa_adjusted_performance['65 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions'] = Lusa_adjusted_performance['62 Platinum_Contribution_Margin_incl_Provisions'] - Lusa_adjusted_performance['64 Acquisition_costs']
    Lusa_adjusted_performance['66 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions'] = Lusa_adjusted_performance['63 Gold_Contribution_Margin_incl_Provisions'] - Lusa_adjusted_performance['64 Acquisition_costs']
    Lusa_adjusted_performance['66a Contribution_Margin_incl_Provisions_w_Acquisitions'] = Lusa_adjusted_performance['63a Contribution_Margin_incl_Provisions'] - Lusa_adjusted_performance['64 Acquisition_costs']

    Lusa_adjusted_performance['67 Platinum_Contribution_Margin_w_Acquisitions'] = Lusa_adjusted_performance['60 Platinum_Contribution_Margin'] - Lusa_adjusted_performance['64 Acquisition_costs']
    Lusa_adjusted_performance['68 Gold_Contribution_Margin_w_Acquisitions'] = Lusa_adjusted_performance['61 Gold_Contribution_Margin'] - Lusa_adjusted_performance['64 Acquisition_costs']
    Lusa_adjusted_performance['68a Contribution_Margin_w_Acquisitions'] = Lusa_adjusted_performance['61a Contribution_Margin'] - Lusa_adjusted_performance['64 Acquisition_costs']

    Lusa_adjusted_performance['69 Platinum_Contribution_Margin_w_Acquisitions_cum'] = Lusa_adjusted_performance.sort_values(['lusa_band', 'mth']).groupby(['lusa_band'])['67 Platinum_Contribution_Margin_w_Acquisitions'].cumsum()
    Lusa_adjusted_performance['70 Gold_Contribution_Margin_w_Acquisitions_cum'] = Lusa_adjusted_performance.sort_values(['lusa_band', 'mth']).groupby(['lusa_band'])['68 Gold_Contribution_Margin_w_Acquisitions'].cumsum()
    Lusa_adjusted_performance['70a Contribution_Margin_w_Acquisitions_cum'] = Lusa_adjusted_performance.sort_values(['lusa_band', 'mth']).groupby(['lusa_band'])['68a Contribution_Margin_w_Acquisitions'].cumsum()

    Lusa_adjusted_performance['71 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions_cum'] = Lusa_adjusted_performance.sort_values(['lusa_band', 'mth']).groupby(['lusa_band'])['65 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions'].cumsum()
    Lusa_adjusted_performance['72 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions_cum'] = Lusa_adjusted_performance.sort_values(['lusa_band', 'mth']).groupby(['lusa_band'])['66 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions'].cumsum()
    Lusa_adjusted_performance['72a Contribution_Margin_incl_Provisions_w_Acquisitions_cum'] = Lusa_adjusted_performance.sort_values(['lusa_band', 'mth']).groupby(['lusa_band'])['66a Contribution_Margin_incl_Provisions_w_Acquisitions'].cumsum()

    Lusa_adjusted_performance_summ = Lusa_adjusted_performance.groupby(['mth']).sum()[Summ_variables]
    Lusa_adjusted_performance_summ['Type'] = 'Forecast - Lusa Normalized'
    # Lusa_adjusted_performance_summ['Vintages Included'] = 'All'
    # return Lusa_adjusted_performance
    return Lusa_adjusted_performance_summ


## Variable lists to be used

In [21]:
# Variables to include in the final version


Variables = ['01 Purchase_gmv_per_released',
 '02 Purchase_igmv_per_released',
 '03 Purchase_igmv_domestic_per_released',
#  '04 Platinum_Gross_domestic_Interchange_revenue_per_released',
#  '05 Platinum_Taxes_domestic_Interchange_revenue_per_released',
#  '06 Platinum_Net_domestic_Interchange_revenue_per_released',
#  '07 Gold_Gross_domestic_Interchange_revenue_per_released',
#  '08 Gold_Taxes_domestic_Interchange_revenue_per_released',
#  '09 Gold_Net_domestic_Interchange_revenue_per_released',
 '07a Gross_domestic_Interchange_revenue_per_released',
 '08a Taxes_domestic_Interchange_revenue_per_released',
 '09a Net_domestic_Interchange_revenue_per_released',
 '10 Purchase_igmv_international_per_released',
 '11 Purchase_igmv_international_interchange_per_released',
 '12 Purchase_igmv_international_Taxes_interchange_per_released',
 '13 Net_international_interchange_per_released',
 '14 fx_spread_per_released',
 '15 fx_spread_taxes_per_released',
 '16 net_fx_spread_per_released',
 '17 net_fx_rate',
 '18 MC_spend_linked_domestic_per_released',
 '19 MC_spend_linked_international_per_released',
 '20 MC_spend_linked_costs_per_released',
 '21 MC_spend_linked_costs_rate',
#  '22 Platinum_Net_non_interest_income_per_released',
#  '23 Gold_Net_non_interest_income_per_released',
 '23a Net_non_interest_income_per_released',
#  '24 Platinum_Interchange_rate',
#  '25 Gold_Interchange_rate',
 '25a Interchange_rate',
#  '26 Platinum_MC_account_linked_cost_per_released',
#  '27 Gold_MC_account_linked_cost_per_released',
 '27a MC_account_linked_cost_per_released',
 '28 Conductor_cost_per_released',
 '29 Partner_Bank_Bill_charging_cost_per_released',
 '30 Bracce_cost_per_released',
 '31 Sorocred_cost_per_released',
 '32 Reissunace_cost_per_released',
 '33 Collections_cost_per_released',
 '34 Cust_Mgmt_cost_per_released',
 '35 Tech_Comm_cost_per_released',
#  '36 Platinum_Non_Interest_Expense_per_released',
#  '37 Gold_Non_Interest_Expense_per_released',
 '37a Non_Interest_Expense_per_released',
 '38 ADB_per_released', '38 ADB_late_per_released', '38 ADB_revolve_per_released', '38 ADB_installments_per_released',
 '38 ADB_other_per_released',
 '38a B_interest_per_released',
 '38b B_lfee_per_released',
 '38c B_reward_fee_per_released',
 '39 Billed_interest_per_released',
 '40 Tax_on_interest_per_released',
 '41 Interest_Income_on_balances_per_released',
 '42 Post_Tax_Interest_Rate',
 '43 Interest_Expense_per_released',
 '44 Interest_Expense_per_ADB',
 '45 Net_Interest_Income_per_released',
 '46 Max_Lines_per_released',
 '47 Chargeoffs_per_released',
 '48 Chargeoffs_Lines_per_released',
 '49 Losses_per_released',
 '50 Recognized_Credit_Losses_per_released',
 '51 Perfect_provisions_per_released',
 '51 absolute_provisions_per_released',
 '52 Change_in_provisions_per_released',
 '52a EOM outstanding_balance_per_released',
 '52b Avg outstanding_balance_per_released',
 '53 Transaction_fraud_per_released',
 '54 ID_fraud_per_released',
 '54a Recovery_per_released',
 '55 Recognized_Losses_per_released',
#  '56 Platinum_Risk_Adjusted_Margin',
#  '57 Gold_Risk_Adjusted_Margin',
 '57a Risk_Adjusted_Margin',
#  '58 Platinum_Risk_Adjusted_Margin_incl_Provisions',
#  '59 Gold_Risk_Adjusted_Margin_incl_Provisions',
 '59a Risk_Adjusted_Margin_incl_Provisions',
#  '60 Platinum_Contribution_Margin',
#  '61 Gold_Contribution_Margin',
 '61a Contribution_Margin',
#  '62 Platinum_Contribution_Margin_incl_Provisions',
#  '63 Gold_Contribution_Margin_incl_Provisions',
 '63a Contribution_Margin_incl_Provisions',
 '64 Acquisition_costs',
#  '65 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions',
#  '66 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions',
 '66a Contribution_Margin_incl_Provisions_w_Acquisitions',
#  '67 Platinum_Contribution_Margin_w_Acquisitions',
#  '68 Gold_Contribution_Margin_w_Acquisitions',
 '68a Contribution_Margin_w_Acquisitions',
#  '69 Platinum_Contribution_Margin_w_Acquisitions_cum',
#  '70 Gold_Contribution_Margin_w_Acquisitions_cum',
 '70a Contribution_Margin_w_Acquisitions_cum','71 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions_cum','72 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions_cum','72a Contribution_Margin_incl_Provisions_w_Acquisitions_cum','Type','Vintages Included']

In [22]:
# Variables to include in the final version


Summ_variables = ['01 Purchase_gmv_per_released',
 '02 Purchase_igmv_per_released',
 '03 Purchase_igmv_domestic_per_released',
#  '04 Platinum_Gross_domestic_Interchange_revenue_per_released',
#  '05 Platinum_Taxes_domestic_Interchange_revenue_per_released',
#  '06 Platinum_Net_domestic_Interchange_revenue_per_released',
#  '07 Gold_Gross_domestic_Interchange_revenue_per_released',
#  '08 Gold_Taxes_domestic_Interchange_revenue_per_released',
#  '09 Gold_Net_domestic_Interchange_revenue_per_released',
 '07a Gross_domestic_Interchange_revenue_per_released',
 '08a Taxes_domestic_Interchange_revenue_per_released',
 '09a Net_domestic_Interchange_revenue_per_released',
 '10 Purchase_igmv_international_per_released',
 '11 Purchase_igmv_international_interchange_per_released',
 '12 Purchase_igmv_international_Taxes_interchange_per_released',
 '13 Net_international_interchange_per_released',
 '14 fx_spread_per_released',
 '15 fx_spread_taxes_per_released',
 '16 net_fx_spread_per_released',
 '17 net_fx_rate',
 '18 MC_spend_linked_domestic_per_released',
 '19 MC_spend_linked_international_per_released',
 '20 MC_spend_linked_costs_per_released',
 '21 MC_spend_linked_costs_rate',
 #'22 Platinum_Net_non_interest_income_per_released',
 #'23 Gold_Net_non_interest_income_per_released',
 '23a Net_non_interest_income_per_released',
#  '24 Platinum_Interchange_rate',
#  '25 Gold_Interchange_rate',
 '25a Interchange_rate',
#  '26 Platinum_MC_account_linked_cost_per_released',
#  '27 Gold_MC_account_linked_cost_per_released',
 '27a MC_account_linked_cost_per_released',
 '28 Conductor_cost_per_released',
 '29 Partner_Bank_Bill_charging_cost_per_released',
 '30 Bracce_cost_per_released',
 '31 Sorocred_cost_per_released',
 '32 Reissunace_cost_per_released',
 '33 Collections_cost_per_released',
 '34 Cust_Mgmt_cost_per_released',
 '35 Tech_Comm_cost_per_released',
#  '36 Platinum_Non_Interest_Expense_per_released',
#  '37 Gold_Non_Interest_Expense_per_released',
 '37a Non_Interest_Expense_per_released',
 '38 ADB_per_released',
 '38a B_interest_per_released',
 '38b B_lfee_per_released',
# '38c B_reward_fee_per_released',
 '39 Billed_interest_per_released',
 '40 Tax_on_interest_per_released',
 '41 Interest_Income_on_balances_per_released',
 '42 Post_Tax_Interest_Rate',
 '43 Interest_Expense_per_released',
 '44 Interest_Expense_per_ADB',
 '45 Net_Interest_Income_per_released',
 '46 Max_Lines_per_released',
 '47 Chargeoffs_per_released',
 '48 Chargeoffs_Lines_per_released',
 '49 Losses_per_released',
 '50 Recognized_Credit_Losses_per_released',
 '51 Perfect_provisions_per_released',
 '52 Change_in_provisions_per_released',
 '53 Transaction_fraud_per_released',
 '54 ID_fraud_per_released',
 '54a Recovery_per_released',
 '55 Recognized_Losses_per_released',
#  '56 Platinum_Risk_Adjusted_Margin',
#  '57 Gold_Risk_Adjusted_Margin',
 '57a Risk_Adjusted_Margin',
#  '58 Platinum_Risk_Adjusted_Margin_incl_Provisions',
#  '59 Gold_Risk_Adjusted_Margin_incl_Provisions',
 '59a Risk_Adjusted_Margin_incl_Provisions',
#  '60 Platinum_Contribution_Margin',
#  '61 Gold_Contribution_Margin',
 '61a Contribution_Margin',
#  '62 Platinum_Contribution_Margin_incl_Provisions',
#  '63 Gold_Contribution_Margin_incl_Provisions',
 '63a Contribution_Margin_incl_Provisions',
 '64 Acquisition_costs',
#  '65 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions',
#  '66 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions',
 '66a Contribution_Margin_incl_Provisions_w_Acquisitions',
#  '67 Platinum_Contribution_Margin_w_Acquisitions',
#  '68 Gold_Contribution_Margin_w_Acquisitions',
 '68a Contribution_Margin_w_Acquisitions',
#  '69 Platinum_Contribution_Margin_w_Acquisitions_cum',
#  '70 Gold_Contribution_Margin_w_Acquisitions_cum',
 '70a Contribution_Margin_w_Acquisitions_cum',
#  '71 Platinum_Contribution_Margin_incl_Provisions_w_Acquisitions_cum',
#  '72 Gold_Contribution_Margin_incl_Provisions_w_Acquisitions_cum',
 '72a Contribution_Margin_incl_Provisions_w_Acquisitions_cum']

## FINAL STEP

In [23]:
# TODO DOUBLE CHECK THOSE CONSTANTS - ESPECIALLY CARD LINKED COSTS (SHOULD HAVE 25% DISC?)
PLATINUM_DOMESTIC_INTERCHANGE_RATE = 0.0165
GOLD_DOMESTIC_INTERCHANGE_RATE = 0.015
FOREIGN_INTERCHANGE_RATE = 0.0175  # not splitting platinum vs gold
OPERATING_REVENUES_TAXES = 0.1125 # includes ISS, PIS and Cofins
FINANCIAL_REVENUES_TAXES = 0.0465 # ISS does not apply, includes PIS and Cofins
MC_SPEND_LINKED_DOMESTIC_RATE = 0.75 * 0.0024
MC_SPEND_LINKED_FOREIGN_RATE = 0.75 * 0.0223
MC_PLATINUM_CARD_LINKED_COST = 4.2
MC_GOLD_CARD_LINKED_COST = 0.6
COST_OF_DEBT_ANNUAL = 0.27 # Does not assume GS cost of debt grossed-up for taxes -> Should probably switch to ~0.32
COST_OF_DEBT_MONTHLY = ((1 + COST_OF_DEBT_ANNUAL)**(1.0/12)) - 1

CONDUCTOR_COST_PER_RELEASED_SCALE = 0
CONDUCTOR_COST_PER_RELEASED = 1.1
REISSUANCE_COST_PER_RELEASED = 0
COLLECTIONS_COST_PER_RELEASED = 0
CUST_MGMT_COST_PER_RELEASED = 2.3
TECH_COMM_COST_PER_RELEASED = 0

ACQUISITION_COSTS_PER_RELEASED = 3.5

# TODO UPDATE PROVISION TABLE
PROVISION = {} # FAIRWAY PROVISION TABLE (BASED ON ROLL-RATES) TO BE SYNCED WITH FINANCE ON A MONTHLY / QUARTERLY BASIS
PROVISION['current'] = 0.0077
PROVISION['revolving'] = 0.041
PROVISION['0-30'] = 0.2522
PROVISION['30-60'] = 0.6417
PROVISION['60-90'] = 0.8557
PROVISION['90-120'] = 0.9252
PROVISION['120-150'] = 0.9611
PROVISION['150-180'] = 0.9873
PROVISION['180+'] = 1.0000

In [26]:
map_for_vintages_included = {0: 'One single month', 1: 'All', 2: 'Last 12 months', 3: 'Last 6 months', 4: 'Last 3 months'}
Final = pd.DataFrame()
cust_counts = pd.DataFrame()

base_line_lusa = 'rule 600+'
Lusa_cutoff = 600
Atscale_costs = "No"
Platinum_Gold = "None"
Scenario = "Lusa_Cutoff = " + str(Lusa_cutoff) + " || At Scale Costs = " + Atscale_costs + "  || Platinum or Gold = " + Platinum_Gold

for j in range(0, cohort_list['code'].max() + 2, 1):

    print j
    if j == 0:

        for i in range(0, cohort_list.level_0.max() + 1, 1):
            ca = np.array(cohort_list[cohort_list.level_0 == i].cohort)
            print ca
            Cohort_metrics, Cust_Lusa_dist = cm(ca, summ4, Lusa_cutoff, Atscale_costs, Platinum_Gold)

            # labeling the vintage from the list for which this was run
            Cohort_metrics['Vintages Included'] = str(ca[0])
            print '--------'
            Lusa_adjusted_performance_summ = lap(ledgers2, Cust_Lusa_dist, Summ_variables)

            # labeling the vintage from the list for which this was run
            Lusa_adjusted_performance_summ['Vintages Included'] = str(ca[0])

            # remains unchanged across loop runs
            Lusa_adjusted_performance_summ650 = lapa(ledgers2, Summ_variables, base_line_lusa)

            # labeling the vintage from the list for which this was run
            Lusa_adjusted_performance_summ650['Vintages Included'] = str(ca[0])
            Lusa_adjusted_performance_summ1 = Lusa_adjusted_performance_summ.reset_index().append(
                Lusa_adjusted_performance_summ650.reset_index())
            Final = Final.append(Lusa_adjusted_performance_summ1.append(Cohort_metrics[Variables].reset_index()))
            cust_temp = Cohort_metrics[Cohort_metrics.reset_index().mth == 0][['cust']]
            cust_temp['Vintages Included'] = str(ca[0])
            cust_counts = cust_counts.append(cust_temp)



    elif j <= 4:

        ca = np.array(cohort_list[cohort_list.code >= j].cohort)
        print ca
        Cohort_metrics, Cust_Lusa_dist = cm(ca, summ4, Lusa_cutoff, Atscale_costs, Platinum_Gold)

        # labeling non monthly vintages
        Cohort_metrics['Vintages Included'] = map_for_vintages_included[j]
        print '--------'
        Lusa_adjusted_performance_summ = lap(ledgers2, Cust_Lusa_dist, Summ_variables)

        # labeling non monthly vintages
        Lusa_adjusted_performance_summ['Vintages Included'] = map_for_vintages_included[j]
        Lusa_adjusted_performance_summ650 = lapa(ledgers2, Summ_variables,base_line_lusa)

        # labeling non monthly vintages
        Lusa_adjusted_performance_summ650['Vintages Included'] = map_for_vintages_included[j]
        Lusa_adjusted_performance_summ1 = Lusa_adjusted_performance_summ.reset_index().append(
            Lusa_adjusted_performance_summ650.reset_index())
        Final = Final.append(Lusa_adjusted_performance_summ1.append(Cohort_metrics[Variables].reset_index()))
        cust_temp = Cohort_metrics[Cohort_metrics.reset_index().mth == 0][['cust']]
        cust_temp['Vintages Included'] = map_for_vintages_included[j]
        cust_counts = cust_counts.append(cust_temp)


    else:

        for k in range(0, cohort_list3.qtr_index.max() + 1, 1):
            ca = np.array(cohort_list3[cohort_list3.qtr_index == k].cohort)
            print ca
            Cohort_metrics, Cust_Lusa_dist = cm(ca, summ4, Lusa_cutoff, Atscale_costs, Platinum_Gold)
            Cohort_metrics['Vintages Included'] = str(
                cohort_list3[cohort_list3.qtr_index == k].qtr.value_counts().iloc[:1])[:6]
            z = Cohort_metrics.reset_index().mth.max()
            print k
            print '--------'
            Lusa_adjusted_performance_summ = lap(ledgers2, Cust_Lusa_dist, Summ_variables)
            Lusa_adjusted_performance_summ['Vintages Included'] = str(
                cohort_list3[cohort_list3.qtr_index == k].qtr.value_counts().iloc[:1])[:6]
            Lusa_adjusted_performance_summ650 = lapa(ledgers2, Summ_variables,base_line_lusa)
            Lusa_adjusted_performance_summ650['Vintages Included'] = str(
                cohort_list3[cohort_list3.qtr_index == k].qtr.value_counts().iloc[:1])[:6]
            Lusa_adjusted_performance_summ1 = Lusa_adjusted_performance_summ.reset_index()[
                Lusa_adjusted_performance_summ.reset_index().mth < z - 1].append(
                Lusa_adjusted_performance_summ650.reset_index())
            Final = Final.append(Lusa_adjusted_performance_summ1.append(
                Cohort_metrics[Variables].reset_index()[Cohort_metrics.reset_index().mth < z - 1]))
            cust_temp = Cohort_metrics[Cohort_metrics.reset_index().mth == 0][['cust']]
            cust_temp['Vintages Included'] = str(
                cohort_list3[cohort_list3.qtr_index == k].qtr.value_counts().iloc[:1])[:6]
            cust_counts = cust_counts.append(cust_temp)

# Renaming columns and creating the final file ready for export
Final_summary = Final.set_index(['mth', 'Type', 'Vintages Included']).stack().reset_index().rename(
    columns={'level_3': 'Metric', 0: 'Value'})

Final_summary = Final_summary.merge(cust_counts.reset_index()[['cust', 'Vintages Included']])

Final_summary['Scenario'] = Scenario
Final_summary['Absolute Value'] = Final_summary['cust'] * Final_summary['Value']

0
[201410]
1
--------


/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:81: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

[201411]
1
--------
[201412]
1
--------
[201501]
1
--------
[201502]
1
--------
[201503]
1
--------
[201504]
1
--------
[201505]
1
--------
[201506]
1
--------
[201507]
1
--------
[201508]
1
--------
[201509]
1
--------
[201510]
1
--------
[201511]
1
--------
[201512]
1
--------
[201601]
1
--------
[201602]
1
--------
[201603]
1
--------
[201604]
1
--------
[201605]
1
--------
[201606]
1
--------
[201607]
1
--------
[201608]
1
--------
[201609]
1
--------
[201610]
1
--------
[201611]
1
--------
[201612]
1
--------
[201701]
1
--------
[201702]
1
--------
[201703]
1
--------
[201704]
1
--------
[201705]
1
--------
[201706]
1
--------
[201707]
1
--------
[201708]
1
--------
[201709]
1
--------
[201710]
1
--------
1
[201410 201411 201412 201501 201502 201503 201504 201505 201506 201507
 201508 201509 201510 201511 201512 201601 201602 201603 201604 201605
 201606 201607 201608 201609 201610 201611 201612 201701 201702 201703
 201704 201705 201706 201707 201708 201709 201710]
37
--------
2


In [27]:
Final_summary.to_csv('Cohort_monitoring_Lusav2_daily_provisions_all_updated_losses_600+_Nov17.csv')

In [35]:
MC = Final_summary[(Final_summary['Vintages Included'].isin(['201703','201704','201705','201706','201707','201708','201709'])) & (Final_summary['Metric'] == '70a Contribution_Margin_w_Acquisitions_cum')].groupby(['Type','Vintages Included','mth']).sum()[['Value']].unstack()
MC

Value                        \
mth                                                 0          1          2     
Type                       Vintages Included                                    
Actuals                    201703            -29.579566 -27.229798 -22.406962   
                           201704            -32.140771 -28.995471 -23.481396   
                           201705            -30.047944 -26.629302 -21.011807   
                           201706            -28.132454 -25.239066 -18.589092   
                           201707            -28.410586 -25.027584 -18.943781   
                           201708            -31.954199 -26.665965 -19.764499   
                           201709            -33.487253 -28.290424        NaN   
Forecast - Lusa 650+       201703            -30.873737 -30.268941 -27.947928   
                           201704            -30.873737 -30.268941 -27.947928   
                           201705            -30.873737 -30.268941 -27.947928   
                           201706            -30.873737 -30.268941 -27.947928   
                           201707            -30.873737 -30.268941 -27.947928   
                           201708            -30.873737 -30.268941 -27.947928   
                           201709            -30.873737 -30.268941 -27.947928   
Forecast - Lusa Normalized 201703            -30.582103 -29.486370 -26.645079   
                           201704            -30.596057 -29.531721 -26.716420   
                           201705            -30.564237 -29.450616 -26.573258   
                           201706            -30.624677 -29.618868 -26.862230   
                           201707            -30.836891 -30.162782 -27.757334   
                           201708            -30.883822 -30.281708 -27.951011   
                           201709            -30.874602 -30.256516        NaN   

                                                                               \
mth                                                 3          4          5     
Type                       Vintages Included                                    
Actuals                    201703            -14.750068  -6.638183   3.210785   
                           201704            -16.182074  -7.217379   3.892545   
                           201705            -12.561312  -1.697152   9.480492   
                           201706             -9.056555   1.318957        NaN   
                           201707            -11.305890        NaN        NaN   
                           201708                   NaN        NaN        NaN   
                           201709                   NaN        NaN        NaN   
Forecast - Lusa 650+       201703            -24.087198 -18.749794 -11.959718   
                           201704            -24.087198 -18.749794 -11.959718   
                           201705            -24.087198 -18.749794 -11.959718   
                           201706            -24.087198 -18.749794 -11.959718   
                           201707            -24.087198 -18.749794 -11.959718   
                           201708            -24.087198 -18.749794 -11.959718   
                           201709            -24.087198 -18.749794 -11.959718   
Forecast - Lusa Normalized 201703            -22.249893 -16.376752  -9.056877   
                           201704            -22.346820 -16.492342  -9.178370   
                           201705            -22.130140 -16.190922  -8.780799   
                           201706            -22.542866 -16.731590        NaN   
                           201707            -23.797920        NaN        NaN   
                           201708                   NaN        NaN        NaN   
                           201709                   NaN        NaN        NaN   

                                                                              \
mth                                                 6          7         8     


In [36]:
Lusa_adjusted_performance_summ650_a = lapa(ledgers2, Summ_variables, base_line_lusa)
Lusa_adjusted_performance_summ650_a

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:124: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:125: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:126: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

Metric,01 Purchase_gmv_per_released,02 Purchase_igmv_per_released,03 Purchase_igmv_domestic_per_released,07a Gross_domestic_Interchange_revenue_per_released,08a Taxes_domestic_Interchange_revenue_per_released,09a Net_domestic_Interchange_revenue_per_released,10 Purchase_igmv_international_per_released,11 Purchase_igmv_international_interchange_per_released,12 Purchase_igmv_international_Taxes_interchange_per_released,13 Net_international_interchange_per_released,...,57a Risk_Adjusted_Margin,59a Risk_Adjusted_Margin_incl_Provisions,61a Contribution_Margin,63a Contribution_Margin_incl_Provisions,64 Acquisition_costs,66a Contribution_Margin_incl_Provisions_w_Acquisitions,68a Contribution_Margin_w_Acquisitions,70a Contribution_Margin_w_Acquisitions_cum,72a Contribution_Margin_incl_Provisions_w_Acquisitions_cum,Type
mth,,,,,,,,,,,,,,,,,,,,,
0,302.543032,212.373371,201.754703,3.026321,0.340461,2.685859,10.618669,0.185827,0.020906,0.164921,...,2.526263,-3.599886,-1.973737,-8.099886,28.9,-36.999886,-30.873737,-30.873737,-36.999886,Forecast - Lusa 650+
1,422.076813,337.876347,320.982530,4.814738,0.541658,4.273080,16.893817,0.295642,0.033260,0.262382,...,5.151782,-1.007002,0.604796,-5.553988,0.0,-5.553988,0.604796,-30.268941,-42.553873,Forecast - Lusa 650+
2,449.106800,391.093088,371.538434,5.573077,0.626971,4.946105,19.554654,0.342206,0.038498,0.303708,...,6.915458,-0.167951,2.321012,-4.762397,0.0,-4.762397,2.321012,-27.947928,-47.316270,Forecast - Lusa 650+
3,474.660358,429.428028,407.956627,6.119349,0.688427,5.430923,21.471401,0.375750,0.042272,0.333478,...,8.502090,0.460368,3.860730,-4.180992,0.0,-4.180992,3.860730,-24.087198,-51.497262,Forecast - Lusa 650+
4,500.026422,462.470320,439.346804,6.590202,0.741398,5.848804,23.123516,0.404662,0.045524,0.359137,...,10.025448,0.779378,5.337404,-3.908666,0.0,-3.908666,5.337404,-18.749794,-55.405928,Forecast - Lusa 650+
5,525.739542,493.286200,468.621890,7.029328,0.790799,6.238529,24.664310,0.431625,0.048558,0.383068,...,11.524797,1.235146,6.790076,-3.499575,0.0,-3.499575,6.790076,-11.959718,-58.905503,Forecast - Lusa 650+
6,548.664040,520.279645,494.265663,7.413985,0.834073,6.579912,26.013982,0.455245,0.051215,0.404030,...,12.303996,1.483096,7.524117,-3.296783,0.0,-3.296783,7.524117,-4.435601,-62.202286,Forecast - Lusa 650+
7,571.179709,545.877058,518.583205,7.778748,0.875109,6.903639,27.293853,0.477642,0.053735,0.423908,...,8.854793,2.034818,4.030384,-2.789591,0.0,-2.789591,4.030384,-0.405217,-64.991877,Forecast - Lusa 650+
8,593.253443,570.856811,542.313971,8.134710,0.915155,7.219555,28.542841,0.499500,0.056194,0.443306,...,9.639147,2.736398,4.770862,-2.131886,0.0,-2.131886,4.770862,4.365645,-67.123763,Forecast - Lusa 650+


In [38]:
Lusa_adjusted_performance_summ650_b = lapa(ledgers2, Summ_variables, base_line_lusa)
Lusa_adjusted_performance_summ650_b

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:99: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:100: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/aakashkatdare/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

Metric,01 Purchase_gmv_per_released,02 Purchase_igmv_per_released,03 Purchase_igmv_domestic_per_released,07a Gross_domestic_Interchange_revenue_per_released,08a Taxes_domestic_Interchange_revenue_per_released,09a Net_domestic_Interchange_revenue_per_released,10 Purchase_igmv_international_per_released,11 Purchase_igmv_international_interchange_per_released,12 Purchase_igmv_international_Taxes_interchange_per_released,13 Net_international_interchange_per_released,...,57a Risk_Adjusted_Margin,59a Risk_Adjusted_Margin_incl_Provisions,61a Contribution_Margin,63a Contribution_Margin_incl_Provisions,64 Acquisition_costs,66a Contribution_Margin_incl_Provisions_w_Acquisitions,68a Contribution_Margin_w_Acquisitions,70a Contribution_Margin_w_Acquisitions_cum,72a Contribution_Margin_incl_Provisions_w_Acquisitions_cum,Type
mth,,,,,,,,,,,,,,,,,,,,,
0,301.367466,224.778521,213.539595,3.203094,0.360348,2.842746,11.238926,0.196681,0.022127,0.174555,...,2.832561,-3.119582,-1.667439,-7.619582,28.9,-36.519582,-30.567439,-30.567439,-36.519582,Forecast - Lusa 650+
1,419.589884,348.298394,330.883474,4.963252,0.558366,4.404886,17.414920,0.304761,0.034286,0.270475,...,5.473479,-0.246663,0.927762,-4.792380,0.0,-4.792380,0.927762,-29.639677,-41.311962,Forecast - Lusa 650+
2,444.188277,395.626166,375.844858,5.637673,0.634238,5.003435,19.781308,0.346173,0.038944,0.307228,...,7.145285,0.978468,2.553752,-3.613066,0.0,-3.613066,2.553752,-27.085925,-44.925028,Forecast - Lusa 650+
3,468.797462,430.989319,409.439853,6.141598,0.690930,5.450668,21.549466,0.377116,0.042426,0.334690,...,8.676253,1.688622,4.039312,-2.948319,0.0,-2.948319,4.039312,-23.046614,-47.873347,Forecast - Lusa 650+
4,493.248018,461.926596,438.830267,6.582454,0.740526,5.841928,23.096330,0.404186,0.045471,0.358715,...,10.153133,2.605219,5.471020,-2.076894,0.0,-2.076894,5.471020,-17.575593,-49.950241,Forecast - Lusa 650+
5,519.368703,492.035011,467.433260,7.011499,0.788794,6.222705,24.601751,0.430531,0.048435,0.382096,...,11.636890,3.707135,6.909055,-1.020700,0.0,-1.020700,6.909055,-10.666538,-50.970941,Forecast - Lusa 650+
6,541.162919,517.492183,491.617574,7.374264,0.829605,6.544659,25.874609,0.452806,0.050941,0.401865,...,12.325081,4.391417,7.553635,-0.380028,0.0,-0.380028,7.553635,-3.112903,-51.350969,Forecast - Lusa 650+
7,562.694308,541.699574,514.614595,7.719219,0.868412,6.850807,27.084979,0.473987,0.053324,0.420664,...,9.107206,4.960001,4.292719,0.145514,0.0,0.145514,4.292719,1.179815,-51.205455,Forecast - Lusa 650+
8,583.808869,565.311690,537.046105,8.055692,0.906265,7.149426,28.265584,0.494648,0.055648,0.439000,...,10.064390,6.486325,5.207519,1.629454,0.0,1.629454,5.207519,6.387335,-49.576002,Forecast - Lusa 650+


In [39]:
Lusa_adjusted_performance_summ650_a['70a Contribution_Margin_w_Acquisitions_cum']

mth
0      -30.873737
1      -30.268941
2      -27.947928
3      -24.087198
4      -18.749794
5      -11.959718
6       -4.435601
7       -0.405217
8        4.365645
9        9.616155
10      15.295832
11      20.630904
12      25.775431
13      30.570448
14      35.343487
15      40.183686
16      44.903335
17      49.531927
18      54.216101
19      59.667052
20      65.419714
21      71.477414
22      77.847528
23      84.289079
24      90.993603
25      97.948435
26     105.074205
27     112.258558
28     119.398319
29     126.369987
          ...    
82     534.553934
83     543.503597
84     552.490031
85     561.512915
86     570.571932
87     579.666776
88     588.797145
89     597.962745
90     607.163290
91     616.398499
92     625.668100
93     634.971826
94     644.309416
95     653.680620
96     663.085190
97     672.522887
98     661.713979
99     649.782372
100    637.409989
101    624.837676
102    612.160595
103    599.433873
104    595.153683
105    590.806526
106   

In [40]:
Lusa_adjusted_performance_summ650_b['70a Contribution_Margin_w_Acquisitions_cum']

mth
0      -30.567439
1      -29.639677
2      -27.085925
3      -23.046614
4      -17.575593
5      -10.666538
6       -3.112903
7        1.179815
8        6.387335
9       12.502563
10      19.136050
11      26.023358
12      33.330563
13      40.713968
14      48.071283
15      56.307832
16      64.216062
17      72.352584
18      80.718170
19      89.171859
20      97.599243
21     106.252088
22     115.358169
23     124.696367
24     134.444662
25     144.585496
26     155.043943
27     165.708286
28     176.477326
29     187.128417
          ...    
82     734.080115
83     744.610514
84     755.145575
85     765.685186
86     776.229233
87     786.777605
88     797.330193
89     807.886889
90     818.447587
91     829.012183
92     839.580575
93     850.152310
94     860.726834
95     871.304130
96     881.884352
97     892.467870
98     882.118569
99     870.801277
100    859.093327
101    847.198795
102    835.199098
103    823.141921
104    818.827958
105    814.457312
106   